Analysis: Vladyslav Kavaka (vladyslav.kavaka@med.uni-muenchen.de), Eduardo Beltran (eduardo.beltran@med.uni-muenchen.de)
Insitute of Clinical Neuroimmunology, LMU, Munich

In [40]:
sessionInfo()
set.seed(1)
.libPaths()

R version 4.0.5 (2021-03-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 18.04.6 LTS

Matrix products: default
BLAS/LAPACK: /home/INIM/vladyslav.kavaka/miniconda3/envs/azimuth/lib/libopenblasp-r0.3.17.so

locale:
 [1] LC_CTYPE=C.UTF-8    LC_NUMERIC=C        LC_TIME=C          
 [4] LC_COLLATE=C        LC_MONETARY=C       LC_MESSAGES=C      
 [7] LC_PAPER=C          LC_NAME=C           LC_ADDRESS=C       
[10] LC_TELEPHONE=C      LC_MEASUREMENT=C    LC_IDENTIFICATION=C

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] ggrepel_0.9.1      ggthemes_4.2.4     scales_1.2.1       reshape2_1.4.4    
 [5] presto_1.0.0       irlba_2.3.5        lisi_1.0           plyr_1.8.6        
 [9] matrixStats_0.61.0 data.table_1.14.2  umap_0.2.7.0       uwot_0.1.11       
[13] RANN_2.6.1         class_7.3-20       purrr_0.3.4        ggplot2_3.4.0     
[17] limma_3.46.0       tidyr_1.1.4        Matrix_1.4-0   

[1] "/home/INIM/vladyslav.kavaka/miniconda3/envs/azimuth/lib/R/library"

In [ ]:
library(harmony)
library(symphony)
library(Seurat)
library(dplyr)
library(enrichR)
library(ComplexUpset)
library(devtools)
library(Matrix)
library(tidyr)
library(limma)
library(ggplot2)
library(purrr)
library(class)
library(RANN)
library(stats)
library(uwot)
library(umap)
library(data.table)
library(matrixStats)
library(plyr)
library(lisi)
library(Rcpp)
library(irlba)
library(presto)
library(reshape2)
library(scales)
library(ggthemes)
library(ggrepel)

In [ ]:
set.seed(1)

In [ ]:
buildReferenceFromSeurat <- function(
    obj, assay = 'RNA', verbose = TRUE, save_umap = TRUE, save_uwot_path = NULL
) {
    if(!assay %in% c('RNA', 'SCT')) {
        stop('Only supported assays are RNA or SCT.')
    }
    res <- list()
    ## TODO: check that these objects are all correctly initialized
    res$Z_corr <- t(obj@reductions$harmony@cell.embeddings)
    res$Z_orig <- t(obj@reductions$pca@cell.embeddings)
    message('Saved embeddings')
    
    res$R <- t(obj@reductions$harmony@misc$R)
    message('Saved soft cluster assignments')
    
    if (assay == 'RNA') {
        vargenes_means_sds <- tibble(
            symbol = obj@assays[[assay]]@var.features, 
            mean = Matrix::rowMeans(obj@assays[[assay]]@data[obj@assays[[assay]]@var.features, ])
        )
        vargenes_means_sds$stddev <- rowSDs(
            obj@assays[[assay]]@data[obj@assays[[assay]]@var.features, ], 
            vargenes_means_sds$mean
        )
    } else if (assay == 'SCT') {
        vargenes_means_sds <- tibble(
            symbol = obj@assays[[assay]]@var.features, 
            mean = Matrix::rowMeans(obj@assays[[assay]]@scale.data[obj@assays[[assay]]@var.features, ])
        )
        asdgc = Matrix(obj@assays[[assay]]@scale.data[obj@assays[[assay]]@var.features, ], sparse = TRUE)
        vargenes_means_sds$stddev <- rowSDs(
            asdgc, 
            vargenes_means_sds$mean
        )
    }
    
    res$vargenes_means_sds <- vargenes_means_sds
    message('Saved variable gene information for ', nrow(vargenes_means_sds), ' genes.')
    
    res$loadings <- obj@reductions$pca@feature.loadings
    message('Saved PCA loadings.')
    
    res$meta_data <- obj@meta.data
    message('Saved metadata.')
    
    ## Check UMAP 
    if (save_umap) {
        if (is.null(save_uwot_path)) {
            error('Please provide a valid path to save_uwot_path in order to save uwot model.')
        }
        if (is.null(obj@reductions$umap@misc$model)) {
            error('uwot model not initialiazed in Seurat object. Please do RunUMAP with umap.method=\'uwot\', return.model=TRUE first.')
        }
        res$umap <- obj@reductions$umap@misc$model
        res$save_uwot_path <- save_uwot_path
        if (file.exists(res$save_uwot_path)) {
            file.remove(res$save_uwot_path)    
        }
        uwot::save_uwot(res$umap, save_uwot_path)
    }
    
    ## Build Reference! 
    if (verbose) 
        message("Calculate final L2 normalized reference centroids (Y_cos)")
    res$centroids = t(cosine_normalize_cpp(res$R %*% t(res$Z_corr), 1))
    if (verbose) 
        message("Calculate reference compression terms (Nr and C)")
    res$cache = compute_ref_cache(res$R, res$Z_corr)
    colnames(res$Z_orig) = row.names(res$metadata)
    rownames(res$Z_orig) = paste0("PC_", seq_len(nrow(res$Z_corr)))
    colnames(res$Z_corr) = row.names(res$metadata)
    rownames(res$Z_corr) = paste0("harmony_", seq_len(nrow(res$Z_corr)))
        
    if (verbose) 
        message("Finished nicely.")
    return(res)    
}

environment(buildReferenceFromSeurat) <- environment(symphony::buildReference)

RunHarmony.Seurat <- function(
  object,
  group.by.vars,
  reduction = 'pca',
  dims.use = NULL,
  theta = NULL,
  lambda = NULL,
  sigma = 0.1,
  nclust = NULL,
  tau = 0,
  block.size = 0.05,
  max.iter.harmony = 10,
  max.iter.cluster = 20,
  epsilon.cluster = 1e-5,
  epsilon.harmony = 1e-4,
  plot_convergence = FALSE,
  verbose = TRUE,
  reference_values = NULL,
  reduction.save = "harmony",
  assay.use = 'RNA',
  project.dim = TRUE,
  ...
) {
  if (reduction == "pca") {
    tryCatch(
      embedding <- Seurat::Embeddings(object, reduction = "pca"),
      error = function(e) {
        if (verbose) {
          message("Harmony needs PCA. Trying to run PCA now.")
        }
        tryCatch(
          object <- Seurat::RunPCA(
            object,
            assay = assay.use, verbose = verbose
          ),
          error = function(e) {
            stop("Harmony needs PCA. Tried to run PCA and failed.")
          }
        )
      }
    )
  } else {
    available.dimreduc <- names(methods::slot(object = object, name = "reductions"))
    if (!(reduction %in% available.dimreduc)) {
      stop("Requested dimension reduction is not present in the Seurat object")
    }
    embedding <- Seurat::Embeddings(object, reduction = reduction)
  }
  if (is.null(dims.use)) {
    dims.use <- seq_len(ncol(embedding))
  }
  dims_avail <- seq_len(ncol(embedding))
  if (!all(dims.use %in% dims_avail)) {
    stop("trying to use more dimensions than computed. Rereun dimension reduction
         with more dimensions or run Harmony with fewer dimensions")
  }
  if (length(dims.use) == 1) {
    stop("only specified one dimension in dims.use")
  }
  metavars_df <- Seurat::FetchData(object, group.by.vars)
    
  harmonyObject <- HarmonyMatrix(
    embedding,
    metavars_df,
    group.by.vars,
    FALSE,
    0,
    theta,
    lambda,
    sigma,
    nclust,
    tau,
    block.size,
    max.iter.harmony,
    max.iter.cluster,
    epsilon.cluster,
    epsilon.harmony,
    plot_convergence,
    TRUE,
    verbose,
    reference_values
  )

  harmonyEmbed <- t(as.matrix(harmonyObject$Z_corr))
  rownames(harmonyEmbed) <- row.names(embedding)
  colnames(harmonyEmbed) <- paste0(reduction.save, "_", seq_len(ncol(harmonyEmbed)))

  harmonyClusters <- t(harmonyObject$R)
  rownames(harmonyClusters) <- row.names(embedding)
  colnames(harmonyClusters) <- paste0('R', seq_len(ncol(harmonyClusters)))
  
  suppressWarnings({
    harmonydata <- Seurat::CreateDimReducObject(
      embeddings = harmonyEmbed,
      stdev = as.numeric(apply(harmonyEmbed, 2, stats::sd)),
      assay = assay.use,
      key = reduction.save,
      misc=list(R=harmonyClusters)
    )
  })

  object[[reduction.save]] <- harmonydata
  if (project.dim) {
    object <- Seurat::ProjectDim(
      object,
      reduction = reduction.save,
      overwrite = TRUE,
      verbose = FALSE
    )
  }
  return(object)
}

environment(RunHarmony.Seurat) <- environment(harmony::HarmonyMatrix)

RunUMAP2 <- function (object, reduction.key = "UMAP_", assay = NULL, reduction.model = NULL, 
    return.model = FALSE, umap.method = "uwot", n.neighbors = 30L, 
    n.components = 2L, metric = "cosine", n.epochs = NULL, learning.rate = 1, 
    min.dist = 0.3, spread = 1, set.op.mix.ratio = 1, local.connectivity = 1L, 
    repulsion.strength = 1, negative.sample.rate = 5, a = NULL, 
    b = NULL, uwot.sgd = FALSE, seed.use = 42, metric.kwds = NULL, 
    angular.rp.forest = FALSE, verbose = TRUE, ...) 
{
    CheckDots(...)
    if (!is.null(x = seed.use)) {
        set.seed(seed = seed.use)
    }
    if (umap.method != "umap-learn" && getOption("Seurat.warn.umap.uwot", 
        TRUE)) {
        warning("The default method for RunUMAP has changed from calling Python UMAP via reticulate to the R-native UWOT using the cosine metric", 
            "\nTo use Python UMAP via reticulate, set umap.method to 'umap-learn' and metric to 'correlation'", 
            "\nThis message will be shown once per session", 
            call. = FALSE, immediate. = TRUE)
        options(Seurat.warn.umap.uwot = FALSE)
    }
    if (umap.method == "uwot-learn") {
        warning("'uwot-learn' is deprecated. Set umap.method = 'uwot' and return.model = TRUE")
        umap.method <- "uwot"
        return.model <- TRUE
    }
    if (return.model) {
        if (verbose) {
            message("UMAP will return its model")
        }
        umap.method = "uwot"
    }
    if (inherits(x = object, what = "Neighbor")) {
        object <- list(idx = Indices(object), dist = Distances(object))
    }
    if (!is.null(x = reduction.model)) {
        if (verbose) {
            message("Running UMAP projection")
        }
        umap.method <- "uwot-predict"
    }
    umap.output <- switch(EXPR = umap.method, `umap-learn` = {
        if (!py_module_available(module = "umap")) {
            stop("Cannot find UMAP, please install through pip (e.g. pip install umap-learn).")
        }
        if (!is.null(x = seed.use)) {
            py_set_seed(seed = seed.use)
        }
        if (typeof(x = n.epochs) == "double") {
            n.epochs <- as.integer(x = n.epochs)
        }
        umap_import <- import(module = "umap", delay_load = TRUE)
        umap <- umap_import$UMAP(n_neighbors = as.integer(x = n.neighbors), 
            n_components = as.integer(x = n.components), metric = metric, 
            n_epochs = n.epochs, learning_rate = learning.rate, 
            min_dist = min.dist, spread = spread, set_op_mix_ratio = set.op.mix.ratio, 
            local_connectivity = local.connectivity, repulsion_strength = repulsion.strength, 
            negative_sample_rate = negative.sample.rate, a = a, 
            b = b, metric_kwds = metric.kwds, angular_rp_forest = angular.rp.forest, 
            verbose = verbose)
        umap$fit_transform(as.matrix(x = object))
    }, uwot = {
        if (metric == "correlation") {
            warning("UWOT does not implement the correlation metric, using cosine instead", 
                call. = FALSE, immediate. = TRUE)
            metric <- "cosine"
        }
        if (is.list(x = object)) {
            umap(X = NULL, nn_method = object, n_threads = nbrOfWorkers(), 
                n_components = as.integer(x = n.components), 
                metric = metric, n_epochs = n.epochs, learning_rate = learning.rate, 
                min_dist = min.dist, spread = spread, set_op_mix_ratio = set.op.mix.ratio, 
                local_connectivity = local.connectivity, repulsion_strength = repulsion.strength, 
                negative_sample_rate = negative.sample.rate, 
                a = a, b = b, fast_sgd = uwot.sgd, verbose = verbose, 
                ret_model = return.model)
        } else {
            umap(X = object, n_threads = nbrOfWorkers(), n_neighbors = as.integer(x = n.neighbors), 
                n_components = as.integer(x = n.components), 
                metric = metric, n_epochs = n.epochs, learning_rate = learning.rate, 
                min_dist = min.dist, spread = spread, set_op_mix_ratio = set.op.mix.ratio, 
                local_connectivity = local.connectivity, repulsion_strength = repulsion.strength, 
                negative_sample_rate = negative.sample.rate, 
                a = a, b = b, fast_sgd = uwot.sgd, verbose = verbose, 
                ret_model = return.model)
        }
    }, `uwot-predict` = {
        if (metric == "correlation") {
            warning("UWOT does not implement the correlation metric, using cosine instead", 
                call. = FALSE, immediate. = TRUE)
            metric <- "cosine"
        }
        if (is.null(x = reduction.model) || !inherits(x = reduction.model, 
            what = "DimReduc")) {
            stop("If running projection UMAP, please pass a DimReduc object with the model stored to reduction.model.", 
                call. = FALSE)
        }
        model <- Misc(object = reduction.model, slot = "model")
        if (length(x = model) == 0) {
            stop("The provided reduction.model does not have a model stored. Please try running umot-learn on the object first", 
                call. = FALSE)
        }
        if (is.list(x = object)) {
            uwot::umap_transform(X = NULL, nn_method = object, 
                model = model, n_threads = nbrOfWorkers(), n_epochs = n.epochs, 
                verbose = verbose)
        } else {
            umap_transform(X = object, model = model, n_threads = nbrOfWorkers(), 
                n_epochs = n.epochs, verbose = verbose)
        }
    }, stop("Unknown umap method: ", umap.method, call. = FALSE))
    if (return.model) {
#         umap.output$nn_index <- NULL
        umap.model <- umap.output
        umap.output <- umap.output$embedding
    }
    colnames(x = umap.output) <- paste0(reduction.key, 1:ncol(x = umap.output))
    if (inherits(x = object, what = "dist")) {
        rownames(x = umap.output) <- attr(x = object, "Labels")
    }
    else if (is.list(x = object)) {
        rownames(x = umap.output) <- rownames(x = object$idx)
    }
    else {
        rownames(x = umap.output) <- rownames(x = object)
    }
    umap.reduction <- CreateDimReducObject(embeddings = umap.output, 
        key = reduction.key, assay = assay, global = TRUE)
    if (return.model) {
        Misc(umap.reduction, slot = "model") <- umap.model
    }
    return(umap.reduction)
}


environment(RunUMAP2) <- environment(Seurat:::RunUMAP.default)

mapQuery <- function (exp_query, metadata_query, ref_obj, vars = NULL, verbose = TRUE, 
    do_normalize = TRUE, do_umap = TRUE, sigma = 0.1, return_type = c('symphony', 'Seurat')) 
{
    if (return_type == 'Seurat') {
        que <- Seurat::CreateSeuratObject(
            counts=exp_query,
            meta.data=metadata_query,
            assay='SymphonyQuery'
        )        
    }
    
    if (do_normalize) {
        if (verbose) 
            message("Normalizing")
        exp_query = normalizeData(exp_query, 10000, "log")
    }
    if (verbose) 
        message("Scaling and synchronizing query gene expression")
    idx_shared_genes = which(ref_obj$vargenes$symbol %in% rownames(exp_query))
    shared_genes = ref_obj$vargenes$symbol[idx_shared_genes]
    if (verbose) 
        message("Found ", length(shared_genes), " reference variable genes in query dataset")
    exp_query_scaled = scaleDataWithStats(exp_query[shared_genes, 
        ], ref_obj$vargenes$mean[idx_shared_genes], ref_obj$vargenes$stddev[idx_shared_genes], 
        1)
    exp_query_scaled_sync = matrix(0, nrow = length(ref_obj$vargenes$symbol), 
        ncol = ncol(exp_query))
    exp_query_scaled_sync[idx_shared_genes, ] = exp_query_scaled
    rownames(exp_query_scaled_sync) = ref_obj$vargenes$symbol
    colnames(exp_query_scaled_sync) = colnames(exp_query)
    if (verbose) 
        message("Project query cells using reference gene loadings")
    Z_pca_query = t(ref_obj$loadings) %*% exp_query_scaled_sync
    if (verbose) 
        message("Clustering query cells to reference centroids")
    Z_pca_query_cos = cosine_normalize_cpp(Z_pca_query, 2)
    R_query = soft_cluster(ref_obj$centroids, Z_pca_query_cos, 
        sigma)
    if (verbose) 
        message("Correcting query batch effects")
    if (!is.null(vars)) {
        design = droplevels(metadata_query)[, vars] %>% as.data.frame()
        onehot = design %>% purrr::map(function(.x) {
            if (length(unique(.x)) == 1) {
                rep(1, length(.x))
            }
            else {
                stats::model.matrix(~0 + .x)
            }
        }) %>% purrr::reduce(cbind)
        Xq = cbind(1, intercept = onehot) %>% t()
    }
    else {
        Xq = Matrix(rbind(rep(1, ncol(Z_pca_query)), rep(1, ncol(Z_pca_query))), 
            sparse = TRUE)
    }
    Zq_corr = moe_correct_ref(as.matrix(Z_pca_query), as.matrix(Xq), 
        as.matrix(R_query), as.matrix(ref_obj$cache[[1]]), as.matrix(ref_obj$cache[[2]]))
    colnames(Z_pca_query) = row.names(metadata_query)
    rownames(Z_pca_query) = paste0("PC_", seq_len(nrow(Zq_corr)))
    colnames(Zq_corr) = row.names(metadata_query)
    rownames(Zq_corr) = paste0("harmony_", seq_len(nrow(Zq_corr)))
    umap_query = NULL
    if (do_umap & !is.null(ref_obj$save_uwot_path)) {
        if (verbose) 
            message("UMAP")
        ref_umap_model = uwot::load_uwot(ref_obj$save_uwot_path, 
            verbose = FALSE)
        
        ## UMAP may have been learned on subset of columns
        umap_query = uwot::umap_transform(t(Zq_corr)[, 1:ref_umap_model$norig_col], ref_umap_model)
#         umap_query = uwot::umap_transform(t(Zq_corr), ref_umap_model)
        colnames(umap_query) = c("UMAP1", "UMAP2")
        rownames(umap_query) <- row.names(metadata_query)
    }
    if (verbose) 
        message("All done!")
    
    if (return_type == 'Seurat') {
        que@assays$SymphonyQuery@data <- exp_query
        que@assays$SymphonyQuery@scale.data <- exp_query_scaled_sync
        que[['pca']] <- Seurat::CreateDimReducObject(
            embeddings = t(Z_pca_query),
            loadings = ref_obj$loadings, 
            stdev = as.numeric(apply(Z_pca_query, 1, stats::sd)),
            assay = 'SymphonyQuery',
            key = 'pca_'
        )
        que[['harmony']] <- Seurat::CreateDimReducObject(
            embeddings = t(Zq_corr),
            stdev = as.numeric(apply(Zq_corr, 1, stats::sd)),
            assay = 'SymphonyQuery',
            key = 'harmony_',
            misc=list(R=R_query)
        )
        que <- Seurat::ProjectDim(que, reduction = 'harmony', overwrite = TRUE, verbose = FALSE)
        if (do_umap) {
            que[['umap']] <- Seurat::CreateDimReducObject(
                embeddings = umap_query,
                assay = 'SymphonyQuery',
                key = 'umap_'
            )            
        }
        return(que)
    } else if (return_type == 'symphony') {
        return(list(Z = Zq_corr, Zq_pca = Z_pca_query, R = R_query, 
            Xq = Xq, umap = umap_query, meta_data = metadata_query))
    } else {
        stop(glue('The return type = \"{return_type}\" is not available.'))
    }
    
}

environment(mapQuery) <- environment(symphony::mapQuery)

knnPredict.Seurat <- function(query_obj, ref_obj, label_transfer, k = 5) 
{
    if (!label_transfer %in% colnames(ref_obj$meta_data)) {
        stop('Label \"{label_transfer}\" is not available in the reference metadata.')
    }
    knn_pred <- class::knn(t(ref_obj$Z_corr), Embeddings(query_obj, 'harmony'), 
        ref_obj$meta_data[[label_transfer]], k = k)
    query_obj@meta.data[[label_transfer]] <- knn_pred
    return(query_obj)
}

In [ ]:
options(repr.plot.width=11, repr.plot.height=11)

In [ ]:
set_figsize <- function(width, height){
    options(repr.plot.width = width, 
            repr.plot.height = height)
}

# Prepare data of validation cohort

In [ ]:
## Read 10X data:
matrix_dir = "pathway to the combined counts of validation cohort"

In [ ]:
list.files(matrix_dir)

In [ ]:
data <- Read10X(data.dir = matrix_dir)
## Create Seurat object
pbmc <- CreateSeuratObject (counts = data, min.cells = 3, min.features = 200, project = "PBMC_Dec_2021")

In [ ]:
pbmc

In [ ]:
# The number of features and UMIs (nFeature_RNA and nCount_RNA) are automatically calculated for every object by Seurat.
# For non-UMI data, nCount_RNA represents the sum of the non-normalized values within a cell
# We calculate the percentage of mitochondrial features here and store it in object metadata as `percent.mito`.
# We use raw count data since this represents non-transformed and non-log-normalized counts
# The % of UMI mapping to MT-features is a common scRNA-seq QC metric.
mito.features <- grep(pattern = "^MT-", x = rownames(x = pbmc), value = TRUE)
percent.mito <- Matrix::colSums(x = GetAssayData(object = pbmc, slot = 'counts')[mito.features, ]) / Matrix::colSums(x = GetAssayData(object = pbmc, slot = 'counts'))

In [ ]:
# The [[ operator can add columns to object metadata, and is a great place to stash QC stats
pbmc[['percent.mito']] <- percent.mito
VlnPlot(object = pbmc, features = c("nFeature_RNA", "nCount_RNA", "percent.mito"), ncol = 3, pt.size = 0)

In [ ]:
# FeatureScatter is typically used to visualize feature-feature relationships, but can be used for anything 
# calculated by the object, i.e. columns in object metadata, PC scores etc.
# Since there is a rare subset of cells with an outlier level of high mitochondrial percentage
# and also low UMI content, we filter these as well
FeatureScatter(object = pbmc, feature1 = "nCount_RNA", feature2 = "percent.mito")

In [ ]:
FeatureScatter(object = pbmc, feature1 = "nCount_RNA", feature2 = "nFeature_RNA")

In [ ]:
# We filter out cells that have unique feature counts over 5,000 or less than 500
pbmc <- subset(x = pbmc, subset = nFeature_RNA > 500 & nFeature_RNA < 5000 & percent.mito < '0.15')

In [ ]:
pbmc

In [ ]:
VlnPlot(object = pbmc, features = c("nFeature_RNA", "nCount_RNA", "percent.mito"), ncol = 3, pt.size = 0.00001)

## Create sample information column

In [ ]:
# Get batches based on cell names
samples_batches <- sapply(colnames(GetAssayData(object = pbmc, slot = "counts")),
                      FUN=function(x){substr(x,18,19)})

In [ ]:
# Turn to numbers and add cell names to them
samples_batches <- as.numeric(as.character(samples_batches))
names(samples_batches) <- colnames(GetAssayData(object = pbmc, slot = "counts"))

In [ ]:
sample.effect <- samples_batches

In [ ]:
pbmc <- AddMetaData(pbmc, sample.effect, "sample.effect")

In [ ]:
Idents(pbmc) <- 'sample.effect'
VlnPlot(object = pbmc, features = "nFeature_RNA", pt.size = 0)

## Adding the sample and diagnosis information:

In [ ]:
info <- read.csv(file = './PBMC_info.csv')

In [ ]:
for(i in 1:nrow(pbmc@meta.data)){
    pbmc@meta.data$sample[i] <- filter(info, Number == pbmc@meta.data$sample.effect[i])$Sample
    pbmc@meta.data$diagnosis[i] <- filter(info, Number == pbmc@meta.data$sample.effect[i])$Diagnosis
}

## Adding TCR information

In [ ]:
#First we need to create TCR file for the first sample in the list
tcr <- read.csv(paste(info$TCR_path[1], "/filtered_contig_annotations.csv", sep=""))
tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
tcr <- tcr[!duplicated(tcr$barcode), ]
#choose the columns to keep
tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
#read clonotypes file
clono <- read.csv(paste(info$TCR_path[1], "/clonotypes.csv", sep=""))
tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
#Rename columns
names(tcr)[1] <- "TCR_clonotype_id"
names(tcr)[3] <- 'TCR_chain'
names(tcr)[4] <- 'TCR_v_gene'
names(tcr)[5] <- 'TCR_frequency'
names(tcr)[6] <- 'TCR_cdr3'
#reorder Columns
tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
#correct rownames
rownames(tcr) <- tcr[,1]
tcr[,1] <- NULL
#Split cdr3 column:
tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
tcr[is.na(tcr)] <- "FALSE"
head(tcr)
tcr.combined <- tcr

In [ ]:
#Now we can start a loop for sample 2 till the end:
for (i in 2:nrow(info)){
    tcr <- read.csv(paste(info$TCR_path[i], "/filtered_contig_annotations.csv", sep=""))
    #change barcode numbers according to samples in the loop
    tcr$barcode <- gsub("-1", paste('-', i, sep = ''), tcr$barcode) 
    tcr <- with(tcr, tcr[order(chain, decreasing = TRUE), ]) # place TRB on top before removing duplicates
    tcr <- tcr[!duplicated(tcr$barcode), ]
    #choose the columns to keep
    tcr <- tcr[,c("barcode", "raw_clonotype_id", "chain", 'v_gene')]
    names(tcr)[names(tcr) == "raw_clonotype_id"] <- "clonotype_id"
    #read clonotypes file
    clono <- read.csv(paste(info$TCR_path[i], "/clonotypes.csv", sep=""))
    tcr <- merge(tcr, clono[, c("clonotype_id", "frequency", "cdr3s_aa")])
    #Rename columns
    names(tcr)[1] <- "TCR_clonotype_id"
    names(tcr)[3] <- 'TCR_chain'
    names(tcr)[4] <- 'TCR_v_gene'
    names(tcr)[5] <- 'TCR_frequency'
    names(tcr)[6] <- 'TCR_cdr3'
    #reorder Columns
    tcr <- tcr[, c(2, 1, 3, 4, 5, 6)]
    #correct rownames
    rownames(tcr) <- tcr[,1]
    tcr[,1] <- NULL
    #Split cdr3 column:
    tcr <- separate(data = tcr, col = TCR_cdr3, into = c("TCR1", "TCR2", "TCR3", "TCR4"), sep = "\\;")
    tcr[is.na(tcr)] <- "FALSE"
    tcr.combined <- rbind(tcr.combined, tcr)
}

In [ ]:
#divide in TRA and TRB subset:
for (k in 1:nrow(tcr.combined)){
  if(startsWith(tcr.combined$TCR1[k], 'TRB:')){
    tcr.combined$TCR1B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR1[k], '')
  } else {tcr.combined$TCR1B[k] <- 'FALSE'}
    if(startsWith(tcr.combined$TCR1[k], 'TRA:')){
    tcr.combined$TCR1A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR1[k], '')
  } else {tcr.combined$TCR1A[k] <- 'FALSE'}
  
  if(startsWith(tcr.combined$TCR2[k], 'TRB:')){
    tcr.combined$TCR2B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR2[k], '')
  } else {tcr.combined$TCR2B[k] <- 'FALSE'}
        if(startsWith(tcr.combined$TCR2[k], 'TRA:')){
    tcr.combined$TCR2A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR2[k], '')
  } else {tcr.combined$TCR2A[k] <- 'FALSE'}
  
  if(startsWith(tcr.combined$TCR3[k], 'TRB:')){
    tcr.combined$TCR3B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR3[k], '')
  } else {tcr.combined$TCR3B[k] <- 'FALSE'}
     if(startsWith(tcr.combined$TCR3[k], 'TRA:')){
    tcr.combined$TCR3A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR3[k], '')
  } else {tcr.combined$TCR3A[k] <- 'FALSE'}
  
  if(startsWith(tcr.combined$TCR4[k], 'TRB:')){
    tcr.combined$TCR4B[k] <- sub(pattern = '.*:', x = tcr.combined$TCR4[k], '')
  } else {tcr.combined$TCR4B[k] <- 'FALSE'}
    if(startsWith(tcr.combined$TCR4[k], 'TRA:')){
    tcr.combined$TCR4A[k] <- sub(pattern = '.*:', x = tcr.combined$TCR4[k], '')
  } else {tcr.combined$TCR4A[k] <- 'FALSE'}
}

In [ ]:
tcr.combined$TCR1 <- NULL
tcr.combined$TCR2 <- NULL
tcr.combined$TCR3 <- NULL
tcr.combined$TCR4 <- NULL
head(tcr.combined)
tail(tcr.combined)

In [ ]:
write.csv(tcr.combined, file = './tcr_pbmc_MS_IIH_all.csv')

In [ ]:
pbmc <- AddMetaData(object = pbmc, metadata = tcr.combined)

In [ ]:
md = pbmc@meta.data # First, let's get the meta data
i <- sapply(md, is.factor) # Identify all factor variables in your data
md[i] <- lapply(md[i], as.character) # Convert factors to character variables
md[is.na(md)] <- "FALSE" # Replace NA with "FALSE"
md[i] <- lapply(md[i], as.factor) # Convert character columns back to factors
pbmc@meta.data = md #Insert it back

## Add CD8 information column

In [ ]:
for (i in 1:nrow(pbmc@meta.data)){
    if(pbmc@meta.data$TCR_frequency[i] == 'FALSE'){
        pbmc@meta.data$V6[i] <- 'FALSE'
    } else {pbmc@meta.data$V6[i] <- 'CD8'}
}

In [ ]:
## Correct frequency column after pre-processing

In [ ]:
#subset only T cells
Idents(pbmc) <- 'V6'
tcells <- subset(pbmc, idents = 'CD8')
tcells

In [ ]:
# create a column containing sample effect and clonotype:
for (i in 1:dim(tcells@meta.data)[1]){
        tcells@meta.data$Sample_Clono[i] <- paste(tcells$sample.effect[i], tcells$TCR_clonotype_id[i], sep = '_')
}

In [ ]:
for (i in 1:nrow(tcells@meta.data)){
    tcells@meta.data$TCR_frequency_corrected[i] <- sum(tcells@meta.data$Sample_Clono == tcells@meta.data$Sample_Clono[i])
}

In [ ]:
tcells@meta.data$TCR_frequency_corrected <- as.numeric(tcells@meta.data$TCR_frequency_corrected)
class(tcells@meta.data$TCR_frequency_corrected)
for (i in 1:dim(tcells@meta.data)[1]){
    if (tcells@meta.data$TCR_frequency_corrected[i] > 2){
        tcells@meta.data$TCR_Clono[i] <- paste(tcells$sample.effect[i], tcells$TCR_clonotype_id[i], tcells$TCR_frequency_corrected[i], sep = '_')
    } else {tcells@meta.data$TCR_Clono[i] <- tcells@meta.data$TCR_frequency_corrected[i]}
}

## Adding expanded column

In [ ]:
#add expand column
for (i in 1:nrow(tcells@meta.data)){
    if(tcells@meta.data$TCR_frequency_corrected[i] > 2){
        tcells@meta.data$expand[i] <- 'exp'
    } else {tcells@meta.data$expand[i] <- 'nonexp'}
}

In [ ]:
head(tcells@meta.data)

## Simplify the diagnosis column

In [ ]:
tcells@meta.data$diagnosis_simp <- tcells@meta.data$diagnosis
tcells@meta.data$diagnosis_simp[tcells@meta.data$diagnosis == 'Relapse'] <- 'MS'
tcells@meta.data$diagnosis_simp[tcells@meta.data$diagnosis == 'Remission'] <- 'MS'

In [ ]:
saveRDS(pbmc, file = './pbmc_MS_IIH_with_tcr_20211205.rds')

In [ ]:
saveRDS(tcells, file = './cd8_MS_IIH_20211205_without_umap.rds')

# Load data for the reference

In [ ]:
ref_data <- readRDS(file = 'pathway to the CD8 Twins Study rds')

In [ ]:
query_data <- tcells

In [ ]:
ref_data@meta.data$orig.ident <- 'ref'
query_data@meta.data$orig.ident <- 'query'

In [ ]:
cells_ref <- ref_data@meta.data
cells_query <- query_data@meta.data

In [ ]:
DefaultAssay(ref_data) <- 'RNA'
# Run standard Seurat pipeline with log normalization
obj <- ref_data %>% 
    NormalizeData(normalization.method = "LogNormalize", scale.factor = 10000)

In [ ]:
obj <- FindVariableFeatures(obj, selection.method = "vst", nfeatures = 2000)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^IGLV|^IGKV|^IGHV|^RPL|^RPS|^IGHC|^MT-",  x = rownames(x = obj), value = TRUE)
VariableFeatures(object = obj) <- VariableFeatures(object = obj)[!(VariableFeatures(object = obj)%in%markers.remove)]

In [ ]:
obj <- ScaleData(obj, features = VariableFeatures(object = obj), vars.to.regress = c("nCount_RNA", "percent.mito"))
obj <- RunPCA(obj, features = VariableFeatures(object = obj))

In [ ]:
options(repr.plot.height = 11, repr.plot.width = 11)
ElbowPlot(object = obj, ndims = 50)

## Create harmony embedding needed for subsequent mapping

In [ ]:
obj <- RunHarmony.Seurat(obj, 'sample.effect') #dims.use

In [ ]:
obj <- FindNeighbors(obj, dims = 1:20, reduction  = 'harmony')
obj <- FindClusters(obj, resolution = 0.5) 

In [ ]:
## Currently, Seurat does not let you cache the umap model for future mapping
## Therefore, please use this custom function to learn a saveable UMAP model
obj[['umap']] <- RunUMAP2(Embeddings(obj, 'harmony')[, 1:20], assay='RNA', verbose=FALSE, umap.method='uwot', return.model=TRUE)

In [ ]:
# Plot reference dims 20
options(repr.plot.height = 11, repr.plot.width = 13)
DimPlot(obj, reduction = 'umap', group.by = 'cd8_coded', label = TRUE)
DimPlot(obj, reduction = 'umap', group.by = 'sample.effect', label = TRUE)
options(repr.plot.height = 11, repr.plot.width = 11)

# Mapping

In [ ]:
obj <- readRDS(file = 'cd8_PBMCs_Twins_reference_harmony.rds')
query_data <- readRDS(file = './cd8_MS_IIH_20211205_without_umap.rds')

## Build symphony reference

In [ ]:
Idents(obj) <- 'cd8_coded'

In [ ]:
ref <- buildReferenceFromSeurat(obj, verbose = TRUE, save_umap = TRUE, save_uwot_path = 'cache_symphony.uwot')

## Map Query 

In [ ]:
Map query starting from query counts and metadata

In [ ]:
query <- mapQuery(
    query_data@assays$RNA@counts, 
    query_data@meta.data, 
    ref,
    vars = 'orig.ident', 
    return_type = 'Seurat'
)

### UMAP

In [ ]:
options(repr.plot.height = 11, repr.plot.width = 11)
DimPlot(obj, reduction = 'umap', group.by = 'cd8_coded', label = TRUE)
DimPlot(query, reduction = 'umap') + labs(title = 'Mapped Query')

### Predict clusters 

In [ ]:
query <- knnPredict.Seurat(query, ref, 'cd8_coded')

In [ ]:
options(repr.plot.height = 11, repr.plot.width = 20)
(DimPlot(obj, reduction = 'umap', shuffle = TRUE, group.by = 'cd8_coded', label =T, label.size = 5, repel = T) + labs(title = 'Original Reference (cd8_coded)'))+ 
(DimPlot(query, reduction = 'umap', shuffle = TRUE, group.by = 'cd8_coded', label =T, label.size = 5, repel = T) + labs(title = 'MS_IIH predicted subclusters'))
options(repr.plot.height = 11, repr.plot.width = 11)

In [ ]:
table(query@meta.data$cd8_coded)

In [ ]:
DimPlot(query, reduction = 'umap', shuffle = TRUE, group.by = 'cd8_coded', label =T, label.size = 6, repel = T)
DimPlot(query, reduction = 'umap', shuffle = TRUE, group.by = 'sample.effect', label =T, label.size = 6, repel = T)

In [ ]:
saveRDS(query, file = './cd8_MS_IIH_mapped_20211206.rds')

In [ ]:
pbmc <- query

In [ ]:
colnames(pbmc@meta.data)
head(pbmc@meta.data)
Idents(pbmc) <- 'cd8_coded'
levels(pbmc) <- c('NK-like', 'MAIT', '1_CCR7', '2_NELL2', '3_NT5E', '4_CD82', '5_MAL', '6_GZMK', '7_MX1', '8_CD74', '9_IKZF2', '10_FGFBP2')
options(repr.plot.width=12, repr.plot.height=11)
DimPlot(pbmc, label = T, label.size = 6)

In [ ]:
unique(pbmc@meta.data$sample)
unique(pbmc@meta.data$diagnosis)
unique(pbmc@meta.data$diagnosis_simp)

length(unique(filter(pbmc@meta.data, diagnosis == 'IIH')$sample))
length(unique(filter(pbmc@meta.data, diagnosis == 'Remission')$sample))
length(unique(filter(pbmc@meta.data, diagnosis == 'Relapse')$sample))

length(unique(filter(pbmc@meta.data, diagnosis_simp == 'MS')$sample))

In [ ]:
#create a simplified column for patients merging the 2 samples for MS-6
pbmc@meta.data$samplenumb <- pbmc@meta.data$sample
pbmc@meta.data$samplenumb[pbmc@meta.data$sample == 'Bl_MS-6-1'] <- 'Bl_MS-6'
pbmc@meta.data$samplenumb[pbmc@meta.data$sample == 'Bl_MS-6-2'] <- 'Bl_MS-6'

In [ ]:
pbmc <- NormalizeData(pbmc, normalization.method = "LogNormalize", scale.factor = 10000)

# Identify the matching cells in the object

In [ ]:
csf_md <- read.csv(file = './CSF_information.csv', row.names = 1)
colnames(csf_md)

In [ ]:
unique(csf_md$patient)

In [ ]:
pbmc@meta.data$patient <- sub(x = pbmc@meta.data$samplenumb, pattern = '.*Bl_', replacement = '')
length(unique(pbmc@meta.data$patient))
sum(unique(pbmc@meta.data$patient) %in% csf_md$patient)

In [ ]:
pbmc_md <- pbmc@meta.data

In [ ]:
#subset only the samples with corresponding csf
nrow(pbmc_md)
subset_pbmc <- pbmc_md[pbmc_md$patient %in% csf_md$patient, ]
nrow(subset_pbmc)

In [ ]:
getmode <- function(v) {
   uniqv <- unique(v)
   uniqv[which.max(tabulate(match(v, uniqv)))]
}


#csf overlap meta.data = md, pbmc_overlap_meta.data = subset_pbmc. Search for overlaps:
md <- csf_md


md$barcodes <- rownames(md)
subset_pbmc$barcodes <- rownames(subset_pbmc)
overlaped_cells_pbmc <- c()
overlaped_tcr_frequency_corresponding_csf <- c()
overlaped_tcr_clono_corresponding_csf <- c()

for(i in 1:length(unique(subset_pbmc$patient))){
    sample_csf <- filter(md, patient == unique(subset_pbmc$patient)[i]) #create subset of the csf meta.data with the selected sample in loop
    sample_pbmc <- filter(subset_pbmc, patient == unique(subset_pbmc$patient)[i])  #create subset of the pbmc meta.data with the selected sample in loop    
    for(r in 1:nrow(sample_pbmc)){
        for(c in 1:4){
            #tcr1b
            if((sample_pbmc[r, paste('TCR', c, 'B', sep = '')] != 'FALSE') & (sample_pbmc[r, paste('TCR', c, 'B', sep = '')] %in% sample_csf$TCR1B)){ #search in TCRB of the corresponding sample
               barcodes <- sample_pbmc$barcodes[r]
               overlaped_cells_pbmc <- c(overlaped_cells_pbmc, barcodes)
               tcr_frequency <- getmode(filter(sample_csf, TCR1B == sample_pbmc[r, paste('TCR', c, 'B', sep = '')])$TCR_frequency_corrected)
               overlaped_tcr_frequency_corresponding_csf <- c(overlaped_tcr_frequency_corresponding_csf, tcr_frequency)
               tcr_clono <- getmode(filter(sample_csf, TCR1B == sample_pbmc[r, paste('TCR', c, 'B', sep = '')])$Sample_Clono)
               overlaped_tcr_clono_corresponding_csf <- c(overlaped_tcr_clono_corresponding_csf, tcr_clono) 
            }
            #tcr2b
           if((sample_pbmc[r, paste('TCR', c, 'B', sep = '')] != 'FALSE') & (sample_pbmc[r, paste('TCR', c, 'B', sep = '')] %in% sample_csf$TCR2B)){ #search in TCRB of the corresponding sample
               barcodes <- sample_pbmc$barcodes[r]
               overlaped_cells_pbmc <- c(overlaped_cells_pbmc, barcodes)
               tcr_frequency <- getmode(filter(sample_csf, TCR2B == sample_pbmc[r, paste('TCR', c, 'B', sep = '')])$TCR_frequency_corrected)
               overlaped_tcr_frequency_corresponding_csf <- c(overlaped_tcr_frequency_corresponding_csf, tcr_frequency)
               tcr_clono <- getmode(filter(sample_csf, TCR2B == sample_pbmc[r, paste('TCR', c, 'B', sep = '')])$Sample_Clono)
               overlaped_tcr_clono_corresponding_csf <- c(overlaped_tcr_clono_corresponding_csf, tcr_clono) 
            }
            #tcr3b
            if((sample_pbmc[r, paste('TCR', c, 'B', sep = '')] != 'FALSE') & (sample_pbmc[r, paste('TCR', c, 'B', sep = '')] %in% sample_csf$TCR3B)){ #search in TCRB of the corresponding sample
               barcodes <- sample_pbmc$barcodes[r]
               overlaped_cells_pbmc <- c(overlaped_cells_pbmc, barcodes)
               tcr_frequency <- getmode(filter(sample_csf, TCR3B == sample_pbmc[r, paste('TCR', c, 'B', sep = '')])$TCR_frequency_corrected)
               overlaped_tcr_frequency_corresponding_csf <- c(overlaped_tcr_frequency_corresponding_csf, tcr_frequency)
               tcr_clono <- getmode(filter(sample_csf, TCR3B == sample_pbmc[r, paste('TCR', c, 'B', sep = '')])$Sample_Clono)
               overlaped_tcr_clono_corresponding_csf <- c(overlaped_tcr_clono_corresponding_csf, tcr_clono) 
            }
            #tcr4b
            if((sample_pbmc[r, paste('TCR', c, 'B', sep = '')] != 'FALSE') & (sample_pbmc[r, paste('TCR', c, 'B', sep = '')] %in% sample_csf$TCR4B)){ #search in TCRB of the corresponding sample
               barcodes <- sample_pbmc$barcodes[r]
               overlaped_cells_pbmc <- c(overlaped_cells_pbmc, barcodes)
               tcr_frequency <- getmode(filter(sample_csf, TCR4B == sample_pbmc[r, paste('TCR', c, 'B', sep = '')])$TCR_frequency_corrected)
               overlaped_tcr_frequency_corresponding_csf <- c(overlaped_tcr_frequency_corresponding_csf, tcr_frequency)
               tcr_clono <- getmode(filter(sample_csf, TCR4B == sample_pbmc[r, paste('TCR', c, 'B', sep = '')])$Sample_Clono)
               overlaped_tcr_clono_corresponding_csf <- c(overlaped_tcr_clono_corresponding_csf, tcr_clono) 
            }
        }
    }
}


In [ ]:
overlap_matrix_pbmc <- data.frame(matrix(NA, ncol = 3, nrow = length(overlaped_cells_pbmc)))
colnames(overlap_matrix_pbmc) <- c('barcodes', 'csf_tcr_frequency', 'csf_tcr_clono')

In [ ]:
overlap_matrix_pbmc$barcodes <- overlaped_cells_pbmc
overlap_matrix_pbmc$csf_tcr_frequency <- overlaped_tcr_frequency_corresponding_csf
overlap_matrix_pbmc$csf_tcr_clono <- overlaped_tcr_clono_corresponding_csf

In [ ]:
overlap_matrix_pbmc <- overlap_matrix_pbmc[!duplicated(overlap_matrix_pbmc$barcodes), ]
head(overlap_matrix_pbmc)
nrow(overlap_matrix_pbmc)

In [ ]:
tail(overlap_matrix_pbmc)

In [ ]:
#subset seurat object
subset_pbmc_object <- pbmc
subset_pbmc_object
unique(subset_pbmc_object@meta.data$patient)

In [ ]:
#add the overlap information into the subseted pbmc file:
for(i in 1:nrow(subset_pbmc_object@meta.data)){
    if(rownames(subset_pbmc_object@meta.data)[i] %in% overlap_matrix_pbmc$barcodes){
        subset_pbmc_object@meta.data$overlap[i] <- 'overlap'
        subset_pbmc_object@meta.data$csf_tcr_frequency[i] <- filter(overlap_matrix_pbmc, barcodes == rownames(subset_pbmc_object@meta.data)[i])$csf_tcr_frequency
        subset_pbmc_object@meta.data$csf_tcr_clono[i] <- filter(overlap_matrix_pbmc, barcodes == rownames(subset_pbmc_object@meta.data)[i])$csf_tcr_clono
    } else {
        subset_pbmc_object@meta.data$overlap[i] <- 'FALSE'
        subset_pbmc_object@meta.data$csf_tcr_frequency[i] <- 'FALSE'
        subset_pbmc_object@meta.data$csf_tcr_clono[i] <- 'FALSE'
    }
}
head(subset_pbmc_object@meta.data)

In [ ]:
overlapedcells <- rownames(filter(subset_pbmc_object@meta.data, overlap == 'overlap'))

In [ ]:
write.csv(subset_pbmc_object@meta.data, file = './pbmc_overlaped_cells_md_20220415.csv')

In [ ]:
saveRDS(subset_pbmc_object, file = 'mapped_pbmc_withoverlapdata_20220415.rds')

In [ ]:
table(subset_pbmc_object@meta.data$overlap)

In [ ]:
pbmc_overlap <- subset_pbmc_object

In [ ]:
pbmc <- readRDS(file = './mapped_pbmc_withoverlapdata_20220415.rds')

# Perform markers identification and plotting

In [ ]:
#create the output dir
dir_plots <- paste0('./outs/')
dir.create(dir_plots)

In [ ]:
width <- 11
height <- 8
name <- 'general_cd8_umap_raster'

options(repr.plot.width = width, repr.plot.height = height)
plot <- DimPlot(pbmc, reduction = "umap", label = TRUE, label.size = 7, repel = TRUE, raster = F, pt.size = 0.01) + 
theme(text = element_text(size = 20),
      axis.text = element_text(size = 20),
      legend.text=element_text(size=18))
plot
ggsave(plot, file = paste0(dir_plots, name, '.pdf'), width = width, height = height)

In [ ]:
pbmc

In [ ]:
featurespbmc <- rownames(pbmc)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS", x = rownames(pbmc), value = TRUE)
featurespbmc <- featurespbmc[!(featurespbmc%in%markers.remove)]
pbmc.markers1 <- FindAllMarkers(object = pbmc, only.pos = TRUE, min.pct = 0.25, logfc.threshold = 0.25, features = featurespbmc)

In [ ]:
#sort markers (top 30)
pbmc.markers_sorted <- c()
for (i in 1:length(levels(pbmc.markers1$cluster))){
    pbmc.markers_level <- filter(pbmc.markers1, cluster == levels(pbmc.markers1$cluster)[i])
    pbmc.markers_level <- pbmc.markers_level[order(-pbmc.markers_level$avg_log2FC), ]
    pbmc.markers_level <- pbmc.markers_level[1:30, ]
    pbmc.markers_level <- pbmc.markers_level[!is.na(pbmc.markers_level$avg_log2FC), ]
    pbmc.markers_sorted <- rbind(pbmc.markers_sorted, pbmc.markers_level)
    }
pbmc.markers_sorted_top30 <- pbmc.markers_sorted
#write.csv(pbmc.markers_sorted_top30, file = './cd8_final_markers.csv')

In [ ]:
#sort markers (top 5)
pbmc.markers_sorted <- c()
for (i in 1:length(levels(pbmc.markers1$cluster))){
    pbmc.markers_level <- filter(pbmc.markers1, cluster == levels(pbmc.markers1$cluster)[i])
    pbmc.markers_level <- pbmc.markers_level[order(-pbmc.markers_level$avg_log2FC), ]
    pbmc.markers_level <- pbmc.markers_level[1:5, ]
    pbmc.markers_level <- pbmc.markers_level[!is.na(pbmc.markers_level$avg_log2FC), ]
    pbmc.markers_sorted <- rbind(pbmc.markers_sorted, pbmc.markers_level)
    }
pbmc.markers_sorted_top5 <- pbmc.markers_sorted
pbmc.markers_sorted_top5

In [ ]:
pbmc.markers_sorted_top30 <- read.csv(file = 'cd8_final_markers.csv', row.names = 1)
pbmc.markers_sorted_top30$cluster <- factor(pbmc.markers_sorted_top30$cluster, levels = levels(pbmc))
pbmc.markers_sorted <- c()
for(i in 1:length(levels(pbmc))){
    pbmc.markers_level <- filter(pbmc.markers_sorted_top30, cluster == levels(pbmc.markers_sorted_top30$cluster)[i])
    pbmc.markers_level <- pbmc.markers_level[order(-pbmc.markers_level$avg_log2FC), ]
    pbmc.markers_level <- pbmc.markers_level[1:5, ]
    pbmc.markers_level <- pbmc.markers_level[!is.na(pbmc.markers_level$avg_log2FC), ]
    pbmc.markers_sorted <- rbind(pbmc.markers_sorted, pbmc.markers_level)
}
pbmc.markers_sorted_top5 <- pbmc.markers_sorted

In [ ]:
width <- 20
height <- 8
options(repr.plot.width = width, repr.plot.height = height)
object <- pbmc

levels(object) <- rev(levels(object))
plot <- DotPlot(object, features = unique(pbmc.markers_sorted_top5$gene), dot.scale = 10, cols = c('white', '#D3556E')) + RotatedAxis() +
        theme(
        text = element_text(size = 17),
        axis.text = element_text(size = 17),
        legend.text=element_text(size=17))
plot
ggsave(plot, file = './outs/dotplot_cd8.pdf', width = width, height = height)

## Expansion overview

In [ ]:
class(pbmc@meta.data$TCR_frequency_corrected)

In [ ]:
pbmc@meta.data$expand_new <- 'non-expanded'
pbmc@meta.data$expand_new[pbmc@meta.data$TCR_frequency_corrected > 2] <- 'expanded'
unique(pbmc@meta.data$expand_new)

In [ ]:
width <- 11
height <- 8
options(repr.plot.width = width, repr.plot.height = height)
expanded_cells <- DimPlot(pbmc, reduction = "umap", label = TRUE, label.size = 8, repel = TRUE, group.by = 'expand_new', cols = c('#D3556E', 'lightgrey'), pt.size = 0.6) + 
theme(text = element_text(size = 20),
      axis.text = element_text(size = 20),
      legend.text=element_text(size=20))
expanded_cells
ggsave(expanded_cells, file = paste0(dir_plots, 'expanded_cells_umap.pdf'), width = width, height = height)

In [ ]:
expansion <- c('non-expanded', 'expanded')
object <- pbmc
Idents(object) <- 'expand_new'
levels(object) <- expansion
levels_subgroups <- levels(pbmc)
object@meta.data$cluster_name <- object@meta.data$cd8_coded
object@meta.data$clusters <- object@meta.data$cluster_name
width <- 14
height <- 11
#explore the diagnosis of the cells
subgroups <- unique(object@meta.data$cluster_name)


results <- c()
for(i in 1:length(subgroups)){
    freq_subgroups <- data.frame(matrix(NA, ncol = length(levels_subgroups), nrow = length(expansion)))
    colnames(freq_subgroups) <- c('expand', 'absolute', 'relative', 'subgroups')
    freq_subgroups$expand <- expansion
    for(c in 1:nrow(freq_subgroups)){
        freq_subgroups$absolute[c] <- nrow(filter(object@meta.data, cluster_name == subgroups[i] & expand_new == freq_subgroups$expand[c]))
        freq_subgroups$relative[c] <- 100 * nrow(filter(object@meta.data, cluster_name == subgroups[i] & expand_new == freq_subgroups$expand[c])) /
                                    nrow(filter(object@meta.data, cluster_name == subgroups[i]))
    }
    freq_subgroups$subgroups <- subgroups[i]
    results <- rbind(results, freq_subgroups)
}
 results$subgroups <- factor(results$subgroups, levels = levels_subgroups)
results$clusters <- factor(results$expand, levels = expansion)
 write.csv(results, file = './outs/expanded_per_cluster_frequencies.csv')


options(repr.plot.width=width, repr.plot.height=height)
plot <- ggplot(results, aes(fill=clusters, y=relative, x=subgroups)) +
        geom_bar(position="stack", stat="identity", ) + theme(
        plot.title = element_text(hjust = 0.45),
        text = element_text(size=30),
        panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), axis.line = element_line(colour = "black"),
        axis.text.x = element_text(angle = 55, vjust = 1, hjust=1, colour = 'black')) + ylab('Fraction of expanded cells within the cluster')+ xlab('Samples') +
        scale_fill_manual('legend', values = c('lightgrey', '#D3556E'))
        #scale_fill_viridis(discrete = TRUE)
        #scale_fill_brewer(palette = "Paired")
    print(plot)
    ggsave(plot, file = paste0(dir_plots, 'expanded_bar_plot.pdf'), width = width, height = height)

## Plotting the matched cells

In [ ]:
overlapedcells <- rownames(filter(pbmc@meta.data, overlap == 'overlap'))

In [ ]:
width <- 11
height <- 8
set_figsize(width, height)
plot <- DimPlot(pbmc, reduction = "umap", label = TRUE, label.size = 7, repel = TRUE, cells.highlight = overlapedcells, sizes.highlight = 0.01, pt.size = 0.01,  cols.highlight = '#D3556E') + 
theme(
      text = element_text(size = 20),
      axis.text = element_text(size = 20),
      legend.text=element_text(size=20))
plot
ggsave(plot, file = paste0(dir_plots, 'overlap_cells_umap.pdf'), width = width, height = height)
options(repr.plot.width = 11, repr.plot.height = 11)

In [ ]:
object <- pbmc
object@meta.data$overlap[object@meta.data$overlap == 'FALSE'] <- 'non-overlapping'
object@meta.data$overlap[object@meta.data$overlap == 'overlap'] <- 'overlapping'

expansion <- c('non-overlapping', 'overlapping')
Idents(object) <- 'overlap'
levels(object) <- expansion
levels_subgroups <- levels(pbmc)
object@meta.data$cluster_name <- object@meta.data$cd8_coded
object@meta.data$clusters <- object@meta.data$cluster_name

width <- 14
height <- 11
#explore the diagnosis of the cells
subgroups <- unique(object@meta.data$cluster_name)


results <- c()
for(i in 1:length(subgroups)){
    freq_subgroups <- data.frame(matrix(NA, ncol = length(levels_subgroups), nrow = length(expansion)))
    colnames(freq_subgroups) <- c('expand', 'absolute', 'relative', 'subgroups')
    freq_subgroups$expand <- expansion
    for(c in 1:nrow(freq_subgroups)){
        freq_subgroups$absolute[c] <- nrow(filter(object@meta.data, cluster_name == subgroups[i] & overlap == freq_subgroups$expand[c]))
        freq_subgroups$relative[c] <- 100 * nrow(filter(object@meta.data, cluster_name == subgroups[i] & overlap == freq_subgroups$expand[c])) /
                                    nrow(filter(object@meta.data, cluster_name == subgroups[i]))
    }
    freq_subgroups$subgroups <- subgroups[i]
    results <- rbind(results, freq_subgroups)
}
 results$subgroups <- factor(results$subgroups, levels = levels_subgroups)
results$clusters <- factor(results$expand, levels = expansion)
 write.csv(results, file = './outs/overlap_per_cluster_frequencies.csv')


options(repr.plot.width=width, repr.plot.height=height)
plot <- ggplot(results, aes(fill=clusters, y=relative, x=subgroups)) +
        geom_bar(position="stack", stat="identity", ) + theme(
        plot.title = element_text(hjust = 0.45),
        text = element_text(size=30),
        panel.grid.major = element_blank(), panel.grid.minor = element_blank(),
        panel.background = element_blank(), axis.line = element_line(colour = "black"),
        axis.text.x = element_text(angle = 55, vjust = 1, hjust=1, colour = 'black')) + ylab('Fraction of expanded cells within the cluster')+ xlab('Samples') +
        scale_fill_manual('legend', values = c('lightgrey', '#D3556E'))
        #scale_fill_viridis(discrete = TRUE)
        #scale_fill_brewer(palette = "Paired")
    print(plot)
    ggsave(plot, file = paste0(dir_plots, 'overlap_bar_plot.pdf'), width = width, height = height)

# Create expression matrix for each diagnosis with selection of features and filtering

In [ ]:
pbmc <- NormalizeData(pbmc, normalization.method = "LogNormalize", scale.factor = 10000)
pbmc_overlap <- NormalizeData(pbmc_overlap, normalization.method = "LogNormalize", scale.factor = 10000)

In [ ]:
#cluster of interest
clusters_of_interest <- c('6_GZMK', '8_CD74', '9_IKZF2', '10_FGFBP2')

## Create the matrix for MS features

In [ ]:
Assays(pbmc)

In [ ]:
Sys.time()

In [ ]:
#create the table with expression values, lasts a beat longer
#determine the sample for the diagnosis group 

#search through the features within all samples within the MSgroup
object <- subset(pbmc, idents = clusters_of_interest)
samples <- unique(filter(object@meta.data, diagnosis_simp == "MS")$samplenumb)
genes <- rownames(object)
cutoff_expression <- 0.1
cutoff_cells <- 0.1

   expression <- data.frame(matrix(NA, ncol = length(samples), nrow  = length(genes)))
    colnames(expression) <- samples
    rownames(expression) <- genes
    for(c in 1:ncol(expression)){
        cells <- rownames(object@meta.data[object@meta.data$samplenumb == colnames(expression)[c], ])
        expression_matrix <- object@assays$SymphonyQuery@data[rownames(expression), cells]
        expression_matrix <- data.frame(expression_matrix)
        expression_matrix$max <- apply(expression_matrix, 1, max)
        expression_matrix$expression_min <- cutoff_expression*expression_matrix$max
        expression_matrix$number_above_cutoff <- rowSums(expression_matrix[1:(ncol(expression_matrix)-2)] > expression_matrix$expression_min)
        expression_matrix$final_filter <- 0
        expression_matrix$final_filter[expression_matrix$number_above_cutoff > (cutoff_cells * length(cells))] <- 1
        expression[, c] <- expression_matrix$final_filter
    }
    expression$sum <- rowSums(expression)
    #select the features that are expressed above the cutoffs more than in the half of the samples
    features_all_samples <- rownames(expression[expression$sum > (0.5 * length(samples)), ])

In [ ]:
#create the table with expression values, lasts a beat longer
#determine the sample for the diagnosis group 

#search through the features within the overlapped group
object <- subset(pbmc_overlap, idents = clusters_of_interest)
object <- subset(object, overlap == "overlap")
samples <- unique(filter(object@meta.data, diagnosis_simp == "MS")$samplenumb)
genes <- rownames(object)
cutoff_expression <- 0.1
cutoff_cells <- 0.1

    expression <- data.frame(matrix(NA, ncol = length(samples), nrow  = length(genes)))
    colnames(expression) <- samples
    rownames(expression) <- genes
    for(c in 1:ncol(expression)){
        cells <- rownames(object@meta.data[object@meta.data$samplenumb == colnames(expression)[c], ])
        expression_matrix <- object@assays$SymphonyQuery@data[rownames(expression), cells]
        expression_matrix <- data.frame(expression_matrix)
        expression_matrix$max <- apply(expression_matrix, 1, max)
        expression_matrix$expression_min <- cutoff_expression*expression_matrix$max
        expression_matrix$number_above_cutoff <- rowSums(expression_matrix[1:(ncol(expression_matrix)-2)] > expression_matrix$expression_min)
        expression_matrix$final_filter <- 0
        expression_matrix$final_filter[expression_matrix$number_above_cutoff > (cutoff_cells * length(cells))] <- 1
        expression[, c] <- expression_matrix$final_filter
    }
expression$sum <- rowSums(expression)
    #select the features that are expressed above the cutoffs more than in the half of the samples
    features_overlap_samples <- rownames(expression[expression$sum > (0.5 * length(samples)), ])

In [ ]:
features_MS <- qpcR:::cbind.na(features_all_samples, features_overlap_samples)
colnames(features_MS) <- c("all samples MS", "overlapped samples MS")
write.csv(features_MS, file = "features_MS.csv")

In [ ]:
Sys.time()

## Create the matrix for IIH features

In [ ]:
#create the table with expression values, lasts a beat longer
#determine the sample for the diagnosis group 

#search through the features within all samples within the IIH group
object <- subset(pbmc, idents = clusters_of_interest)
samples <- unique(filter(object@meta.data, diagnosis_simp == "IIH")$samplenumb)
genes <- rownames(object)
cutoff_expression <- 0.1
cutoff_cells <- 0.1

   expression <- data.frame(matrix(NA, ncol = length(samples), nrow  = length(genes)))
    colnames(expression) <- samples
    rownames(expression) <- genes
    for(c in 1:ncol(expression)){
        cells <- rownames(object@meta.data[object@meta.data$samplenumb == colnames(expression)[c], ])
        expression_matrix <- object@assays$SymphonyQuery@data[rownames(expression), cells]
        expression_matrix <- data.frame(expression_matrix)
        expression_matrix$max <- apply(expression_matrix, 1, max)
        expression_matrix$expression_min <- cutoff_expression*expression_matrix$max
        expression_matrix$number_above_cutoff <- rowSums(expression_matrix[1:(ncol(expression_matrix)-2)] > expression_matrix$expression_min)
        expression_matrix$final_filter <- 0
        expression_matrix$final_filter[expression_matrix$number_above_cutoff > (cutoff_cells * length(cells))] <- 1
        expression[, c] <- expression_matrix$final_filter
    }
expression$sum <- rowSums(expression)
    #select the features that are expressed above the cutoffs more than in the half of the samples
    features_IIH <- rownames(expression[expression$sum > (0.5 * length(samples)), ])

In [ ]:
#create the table with expression values, lasts a beat longer
#determine the sample for the diagnosis group 

#search through the features within the overlapped group
object <- subset(pbmc_overlap, idents = clusters_of_interest)
object <- subset(object, overlap == "overlap")
samples <- unique(filter(object@meta.data, diagnosis_simp == "IIH")$samplenumb)
genes <- rownames(object)
cutoff_expression <- 0.1
cutoff_cells <- 0.1

   expression <- data.frame(matrix(NA, ncol = length(samples), nrow  = length(genes)))
    colnames(expression) <- samples
    rownames(expression) <- genes
    for(c in 1:ncol(expression)){
        cells <- rownames(object@meta.data[object@meta.data$samplenumb == colnames(expression)[c], ])
        expression_matrix <- object@assays$SymphonyQuery@data[rownames(expression), cells]
        expression_matrix <- data.frame(expression_matrix)
        expression_matrix$max <- apply(expression_matrix, 1, max)
        expression_matrix$expression_min <- cutoff_expression*expression_matrix$max
        expression_matrix$number_above_cutoff <- rowSums(expression_matrix[1:(ncol(expression_matrix)-2)] > expression_matrix$expression_min)
        expression_matrix$final_filter <- 0
        expression_matrix$final_filter[expression_matrix$number_above_cutoff > (cutoff_cells * length(cells))] <- 1
        expression[, c] <- expression_matrix$final_filter
    }
expression$sum <- rowSums(expression)
    #select the features that are expressed above the cutoffs more than in the half of the samples
    features_overlap_IIH <- rownames(expression[expression$sum > (0.5 * length(samples)), ])

In [ ]:
features_IIH_matrix <- qpcR:::cbind.na(features_IIH, features_overlap_IIH)
colnames(features_IIH_matrix) <- c("all samples IIH", "overlapped samples IIH")
write.csv(features_IIH_matrix, file = "features_IIH.csv")

In [ ]:
Sys.time()

# Intersect and combine all features

In [ ]:
length(features_all_samples)
length(features_overlap_samples)
features_MS_intersect <- Reduce(intersect, list(features_all_samples, features_overlap_samples))
length(features_MS_intersect)

In [ ]:
length(features_IIH)
length(features_overlap_IIH)
features_IIH_intersect <- Reduce(intersect, list(features_IIH, features_overlap_IIH))
length(features_IIH_intersect)

In [ ]:
features_intersect <- qpcR:::cbind.na(features_MS_intersect, features_IIH_intersect)
colnames(features_intersect) <- c("features_ms", "features_IIH")
write.csv(features_intersect, file = "features_intersect.csv")

# Determine the markers and object for analysis

In [ ]:
#cluster of interest
clusters_of_interest <- c('6_GZMK', '8_CD74', '9_IKZF2', '10_FGFBP2')
object <- subset(pbmc, idents = clusters_of_interest) #will be used for plotting
object

#remove one of the doubled samples from the analysis
Idents(object) <- 'sample'
object_analysis_general <- subset(object, idents = 'Bl_MS-6-2', invert = TRUE)
object_analysis_general

In [ ]:
# determine the markers for comparison
markers <- read.csv(file = 'features_intersect.csv', row.names = 1)
markers.remove <- grep(pattern = "^TRAV|^TRBV|^TRGV|^TRDV|^RPL|^RPS|^IGKV|^IGLV|^IGHV|^IGHG|^IGLC|^TRKC|^MT", x = rownames(object), value = TRUE)


features_ms <- markers$features_ms
features_ms <- features_ms[!is.na(features_ms)]
features_ms <- features_ms[!(features_ms%in%markers.remove)]
length(features_ms)


features_IIH <- markers$features_IIH
features_IIH <- features_IIH[!is.na(features_IIH)]
features_IIH <- features_IIH[!(features_IIH%in%markers.remove)]
length(features_IIH)

In [ ]:
#start enrichR
library('enrichR')
setEnrichrSite("Enrichr")
dir.create('outs')

In [ ]:
set.seed(1)

# MS vs IIH analysis

In [ ]:
#prepare the file
object_analysis <- object_analysis_general
object_analysis

## Start for the first partner

In [ ]:
#find first the markers per cluster
partner1 <- 'MS'
partner2 <- 'IIH'
features_1 <- features_ms
features_2 <- features_IIH
clusters_of_interest <- c('6_GZMK', '8_CD74', '9_IKZF2', '10_FGFBP2')
databases_list <- c('GO_Biological_Process_2021', 'Reactome_2016', 'MSigDB_Hallmark_2020')
grouping_de <- 'diagnosis_simp'
logfc.threshold <- 0.05
pvalue <- 0.05

#create output dirs
dir_path <- paste0('./outs/', partner1, '_vs_', partner2, '_NON_pairwise')
dir.create(dir_path)
plots_dir <- paste0(dir_path, '/plots/')
dir.create(plots_dir)
#define colors
colours_diagnosis_groups <- c('#bbbbbb', '#8e2311')
colour1 <- '#8e2311'
colour2 <- '#bbbbbb'

#object - plotting object
Idents(object) <- grouping_de
levels(object) <- c('IIH', 'MS')
object_av <- AverageExpression(object, return.seurat = TRUE, verbose = FALSE)
Idents(object_analysis) <- 'cd8_coded'
levels(object_analysis) <- clusters_of_interest

de_list_partner1 <- c()
for(i in 1:length(clusters_of_interest)){
        object_subset <- subset(object_analysis, idents = clusters_of_interest[i])
        markers_partner1 <- FindMarkers(object_subset, ident.1 = partner1, ident.2 = partner2, group.by = grouping_de, features = features_1, 
                   only.pos = TRUE, logfc.threshold = logfc.threshold, verbose = FALSE)
        markers_partner1$genes <- rownames(markers_partner1)
        markers_partner1$cluster <- clusters_of_interest[i]
        markers_partner1 <- markers_partner1[markers_partner1$p_val_adj < pvalue, ]
        de_list_partner1 <- append(de_list_partner1, list(markers_partner1))
}

#search through the markers
hits <- 0
selected_markers_partner1 <- c()

for(i in 1:length(de_list_partner1)){
    markers_cluster <- de_list_partner1[[i]]
    #determine the numbers of the other samples
    number_of_othersamples <- 1:length(de_list_partner1)
    number_of_othersamples <- number_of_othersamples[number_of_othersamples != i]
    
    #start to search for the markers in other clusters
    for(r in 1:nrow(markers_cluster)){
        gene_to_test <- markers_cluster$genes[r]
        intersect_gene <- filter(markers_cluster, genes == gene_to_test)
        #open the loop for the other samples
        for(o in number_of_othersamples){
           markers_other_cluster <- de_list_partner1[[o]]
           if(gene_to_test %in% markers_other_cluster$genes){
             intersect_gene <- rbind(intersect_gene, filter(markers_other_cluster, genes == gene_to_test))
           }
        }
        #add the result to the final table
        if(nrow(intersect_gene) > hits){
            gene_to_add <- intersect_gene[1, ]
            gene_to_add$avg_log2FC <- mean(intersect_gene$avg_log2FC)
            gene_to_add$max_log2FC <- max(intersect_gene$avg_log2FC)
            gene_to_add$min_log2FC <- min(intersect_gene$avg_log2FC)
            gene_to_add$avg_p_val_adj <- mean(intersect_gene$p_val_adj)
            gene_to_add$cluster <- paste0(intersect_gene$cluster, collapse = ', ')
            selected_markers_partner1 <- rbind(selected_markers_partner1, gene_to_add)
        }
    }
}
selected_markers_partner1 <- selected_markers_partner1[!duplicated(selected_markers_partner1$genes), ]
selected_markers_partner1 <- selected_markers_partner1[order(-selected_markers_partner1$avg_log2FC), ]

#plot the intersected markers for the next verification step
#create the dir for vln plots per partner
plots_partner1 <- paste0(plots_dir, partner1, '/')
dir.create(plots_partner1)
#create ordered heatmap
ordered_genes <- object_av@assays$SymphonyQuery@data[selected_markers_partner1$genes, ] 
ordered_genes <- as.data.frame(ordered_genes)
ordered_genes <- ordered_genes[order(-ordered_genes[, partner1]), ]

#create ordered heatmap
genes_higher <- as.data.frame(ordered_genes[partner1] > ordered_genes[partner2]) #attention!
genes_higher$genes <- rownames(genes_higher)
genes_higher <- genes_higher$genes[genes_higher[[1]]]
#save the genes higher as the partner on average
write.csv(genes_higher, file = paste0(plots_partner1, 'above_the_partner_average.csv'))

#create ordered heatmap
genes_lower <- as.data.frame(ordered_genes[partner1] < ordered_genes[partner2]) #attention!
genes_lower$genes <- rownames(genes_lower)
genes_lower <- genes_lower$genes[genes_lower[[1]]]
#save the genes lower as the partner on average
write.csv(genes_lower, file = paste0(plots_partner1, 'below_the_partner_average.csv'))

#prepare the genes for heatmap
ordered_genes <- rownames(ordered_genes)

#top 20 sorted
if(length(ordered_genes) > 20){
ordered_genes_plot <- ordered_genes[1:20]
} else {ordered_genes_plot <- ordered_genes}
options(repr.plot.width=5, repr.plot.height=12)
heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, slot = 'data', draw.lines = FALSE, size = 7,  angle = 270, hjust = 1, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 23, colour = 'black', face = 'plain'),
         axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) + 
scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
ggsave(heatmap, file = paste0(plots_partner1, 'heatmap_top20_1.pdf'), width = 5, height = 12)

for(g in 1:length(ordered_genes)){
    plot <- VlnPlot(object, features = ordered_genes[g], pt.size = 0.01, cols = colours_diagnosis_groups)
    plot$layers[[2]]$aes_params$alpha <- 0.1
    ggsave(plot, file = paste0(plots_partner1, ordered_genes[g], '.pdf'), width = 6, height = 6)
}

In [ ]:
#now look through the genes and select the ones validated on the full cohort
genes_remove <- genes_lower
selected_markers_partner1 <- selected_markers_partner1[!selected_markers_partner1$genes %in% genes_remove, ]

#create ordered heatmap
ordered_genes <- object_av@assays$SymphonyQuery@data[selected_markers_partner1$genes, ] 
ordered_genes <- as.data.frame(ordered_genes)
ordered_genes <- ordered_genes[order(-ordered_genes[, partner1]), ]
ordered_genes <- rownames(ordered_genes)

#top 20 sorted
if(length(ordered_genes) > 20){
ordered_genes_plot <- ordered_genes[1:20]
} else {ordered_genes_plot <- ordered_genes}
options(repr.plot.width=5, repr.plot.height=12)
heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, slot = 'data', draw.lines = FALSE, size = 7,  angle = 270, hjust = 1, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 23, colour = 'black', face = 'plain'),
         axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) + 
scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
ggsave(heatmap, file = paste0(dir_path, '/', partner1, '_heatmap_top20.pdf'), width = 5, height = 12)

In [ ]:
#start the enrichr PEA for first partner
dir_pea_partner1 <- paste0(dir_path, '/PEA_', partner1, '/')
dir.create(dir_pea_partner1)

for(db in 1:length(databases_list)){
        enriched <- enrichr(selected_markers_partner1$genes, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the positive enriched pathways
        write.csv(enriched, file = paste0(dir_pea_partner1, partner1, '_', databases_list[db], '_positive_', '_pea_list.csv'))
            
        
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_positive <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(partner1,  'positive', databases_list[db]))
            } else {
            plot_positive <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour1) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(partner1,  'positive', databases_list[db]))
        }
        ggsave(plot_positive, file = paste0(dir_pea_partner1, partner1, '_', databases_list[db], '_positive', '.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
        }
        
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, slot = 'data', draw.lines = FALSE, raster = FALSE, group.colors = colours_diagnosis_groups) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_pea_partner1, partner1, '_', databases_list[db], '_HEATMAP_genes_positive', '.pdf'), width = 8, height = 11)
        
        vlnplot <- VlnPlot(object, features = genes_to_plot, stack = TRUE, flip = TRUE, cols = colours_diagnosis_groups, fill.by = "ident") +
                        theme(legend.position = "none",
                          text = element_text(size = 17),
                          axis.text = element_text(size = 17))
        ggsave(vlnplot, file = paste0(dir_pea_partner1, partner1, '_', databases_list[db], '_StackedVIOLIN_genes_positive', '.pdf'), width = 8, height = 11)
        }
    #end of the einrichr loop

## Start for second partner

In [ ]:
# start the same for the second partner
Idents(object_analysis) <- 'cd8_coded'
levels(object_analysis) <- clusters_of_interest
de_list_partner2 <- c()

for(i in 1:length(clusters_of_interest)){
        object_subset <- subset(object_analysis, idents = clusters_of_interest[i])
        markers_partner2 <- FindMarkers(object_subset, ident.1 = partner2, ident.2 = partner1, group.by = grouping_de, features = features_2, 
                                        only.pos = TRUE, logfc.threshold = logfc.threshold)
        markers_partner2$genes <- rownames(markers_partner2)
        markers_partner2$cluster <- clusters_of_interest[i]
        markers_partner2 <- markers_partner2[markers_partner2$p_val_adj < pvalue, ]
        de_list_partner2 <- append(de_list_partner2, list(markers_partner2))
}

#search through the markers
hits <- 1
selected_markers_partner2 <- c()

for(i in 1:length(de_list_partner2)){
    markers_cluster <- de_list_partner2[[i]]
    #determine the numbers of the other samples
    number_of_othersamples <- 1:length(de_list_partner2)
    number_of_othersamples <- number_of_othersamples[number_of_othersamples != i]
    
    #start to search for the markers in other clusters
    for(r in 1:nrow(markers_cluster)){
        gene_to_test <- markers_cluster$genes[r]
        intersect_gene <- filter(markers_cluster, genes == gene_to_test)
        #open the loop for the other samples
        for(o in number_of_othersamples){
           markers_other_cluster <- de_list_partner2[[o]]
           if(gene_to_test %in% markers_other_cluster$genes){
             intersect_gene <- rbind(intersect_gene, filter(markers_other_cluster, genes == gene_to_test))
           }
        }
        #add the result to the final table
        if(nrow(intersect_gene) > hits){
            gene_to_add <- intersect_gene[1, ]
            gene_to_add$avg_log2FC <- mean(intersect_gene$avg_log2FC)
            gene_to_add$max_log2FC <- max(intersect_gene$avg_log2FC)
            gene_to_add$min_log2FC <- min(intersect_gene$avg_log2FC)
            gene_to_add$avg_p_val_adj <- mean(intersect_gene$p_val_adj)
            gene_to_add$cluster <- paste0(intersect_gene$cluster, collapse = ', ')
            selected_markers_partner2 <- rbind(selected_markers_partner2, gene_to_add)
        }
    }
}
selected_markers_partner2 <- selected_markers_partner2[!duplicated(selected_markers_partner2$genes), ]
selected_markers_partner2 <- selected_markers_partner2[order(-selected_markers_partner2$avg_log2FC), ]

#plot the intersected markers for the next verification step
#create the dir for vln plots per partner
plots_partner2 <- paste0(plots_dir, partner2, '/')
dir.create(plots_partner2)
#create ordered heatmap
ordered_genes <- object_av@assays$SymphonyQuery@data[selected_markers_partner2$genes, ] 
ordered_genes <- as.data.frame(ordered_genes)
ordered_genes <- ordered_genes[order(-ordered_genes[, partner2]), ]

#create ordered heatmap
genes_higher <- as.data.frame(ordered_genes[partner2] > ordered_genes[partner1]) #attention!
genes_higher$genes <- rownames(genes_higher)
genes_higher <- genes_higher$genes[genes_higher[[1]]]
#save the genes higher as the partner on average
write.csv(genes_higher, file = paste0(plots_partner2, 'above_the_partner_average.csv'))

#create ordered heatmap
genes_lower <- as.data.frame(ordered_genes[partner2] < ordered_genes[partner1]) #attention!
genes_lower$genes <- rownames(genes_lower)
genes_lower <- genes_lower$genes[genes_lower[[1]]]
#save the genes lower as the partner on average
write.csv(genes_lower, file = paste0(plots_partner2, 'below_the_partner_average.csv'))

#prepare the genes for heatmap
ordered_genes <- rownames(ordered_genes)

#top 20 sorted
if(length(ordered_genes) > 20){
ordered_genes_plot <- ordered_genes[1:20]
} else {ordered_genes_plot <- ordered_genes}
options(repr.plot.width=5, repr.plot.height=12)
heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, slot = 'data', draw.lines = FALSE, size = 7,  angle = 270, hjust = 1, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 23, colour = 'black', face = 'plain'),
         axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) + 
scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
ggsave(heatmap, file = paste0(plots_partner2, 'heatmap_top20_1.pdf'), width = 5, height = 12)

for(g in 1:length(ordered_genes)){
    plot <- VlnPlot(object, features = ordered_genes[g], pt.size = 0, cols = colours_diagnosis_groups)
    ggsave(plot, file = paste0(plots_partner2, ordered_genes[g], '.pdf'), width = 6, height = 6)
}

In [ ]:
#now look through the genes and select the ones validated on the full cohort

genes_remove <- genes_lower
selected_markers_partner2 <- selected_markers_partner2[!selected_markers_partner2$genes %in% genes_remove, ]

#create ordered heatmap
ordered_genes <- object_av@assays$SymphonyQuery@data[selected_markers_partner2$genes, ] 
ordered_genes <- as.data.frame(ordered_genes)
ordered_genes <- ordered_genes[order(-ordered_genes[, partner2]), ]
ordered_genes <- rownames(ordered_genes)

#top 20 sorted
if(length(ordered_genes) > 20){
ordered_genes_plot <- ordered_genes[1:20]
} else {ordered_genes_plot <- ordered_genes}
options(repr.plot.width=5, repr.plot.height=12)
heatmap <- DoHeatmap(object_av, features = ordered_genes_plot, slot = 'data', draw.lines = FALSE, size = 7,  angle = 270, hjust = 1, raster = FALSE, group.colors = colours_diagnosis_groups) + 
    theme(
        text = element_text(size = 23, colour = 'black', face = 'plain'),
         axis.text.y = element_text(size = 19, colour = 'black', face = 'plain')) + 
scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
ggsave(heatmap, file = paste0(dir_path, '/', partner2, '_heatmap_top20.pdf'), width = 5, height = 12)

In [ ]:
#start the enrichr PEA for first partner
dir_pea_partner2 <- paste0(dir_path, '/PEA_', partner2, '/')
dir.create(dir_pea_partner2)

for(db in 1:length(databases_list)){
        enriched <- enrichr(selected_markers_partner2$genes, databases = databases_list[db])
        enriched <- enriched[[1]]
        enriched <- enriched[order(-enriched$Adjusted.P.value), ]
        enriched$Term <- factor(enriched$Term, levels = unique(enriched$Term))
        #save the positive enriched pathways
        write.csv(enriched, file = paste0(dir_pea_partner2, partner2, '_', databases_list[db], '_positive_', '_pea_list.csv'))
            
        
        reverselog_trans <- function(base = exp(1)) {
            trans <- function(x) -log(x, base)
            inv <- function(x) base^(-x)
            trans_new(paste0("reverselog-", format(base)), trans, inv,
                      log_breaks(base = base),
                      domain = c(1e-100, Inf))
            }
        #2881c1 - for blue
        #d3556e - for red
        options(repr.plot.width=22, repr.plot.height=11)
        if(nrow(enriched) > 20){
            plot_positive <- ggplot(enriched[(nrow(enriched)-19):nrow(enriched), ], aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(partner2,  'positive', databases_list[db]))
            } else {
            plot_positive <- ggplot(enriched, aes(y=Term, x= Adjusted.P.value))+
                geom_vline(xintercept = .05, color = "grey", linetype="dashed") +
                geom_segment( aes(yend=Term, xend=1), col= "black") +
                geom_point(shape=21, aes(size = abs(Combined.Score)), fill = colour2) +
                scale_x_continuous(trans=reverselog_trans(10))+
                scale_size_continuous(range = c(2, 13)) +
                theme_tufte()+ xlab("p value (adj)") + ylab("") +
                theme(text=element_text(family="Helvetica", size = 26),
                  axis.text.x = element_text(angle = 45, hjust = 1), legend.position = "right") + ggtitle(paste(partner2,  'positive', databases_list[db]))
        }
        ggsave(plot_positive, file = paste0(dir_pea_partner2, partner2, '_', databases_list[db], '_positive', '.pdf'), width = 30, height = 11)
            
        #plot the genes
        if(nrow(enriched) > 20){ 
            genes_to_plot <- paste0(x = enriched[(nrow(enriched)-19):nrow(enriched), 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            #reverse because of ascending ordering of the enriched table to put the most significant at the beginning
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
            }else{
            genes_to_plot <- paste0(x = enriched[, 'Genes'], ';')
            genes_to_plot <- paste(genes_to_plot, collapse = '')
            genes_to_plot <- strsplit(genes_to_plot, split = ';')[[1]]
            genes_to_plot <- rev(genes_to_plot)
            genes_to_plot <- unique(genes_to_plot)
        }
        
        if(length(genes_to_plot) > 20){genes_to_plot <- genes_to_plot[1:20]}
        heatmap_plot <- DoHeatmap(object_av, features = genes_to_plot, slot = 'data', draw.lines = FALSE, raster = FALSE, group.colors = colours_diagnosis_groups) + theme(text = element_text(size = 20, face = "bold")) + scale_fill_gradientn(colors = c("#2881C1", "white", "#D3556E"))
        ggsave(heatmap_plot, file = paste0(dir_pea_partner2, partner2, '_', databases_list[db], '_HEATMAP_genes_positive', '.pdf'), width = 8, height = 11)
        
        vlnplot <- VlnPlot(object, features = genes_to_plot, stack = TRUE, flip = TRUE, cols = colours_diagnosis_groups, fill.by = "ident") +
                        theme(legend.position = "none",
                          text = element_text(size = 17),
                          axis.text = element_text(size = 17))
        ggsave(vlnplot, file = paste0(dir_pea_partner2, partner2, '_', databases_list[db], '_StackedVIOLIN_genes_positive', '.pdf'), width = 8, height = 11)
        }
    #end of the einrichr loop

## Combine the final result and plot volcano

In [ ]:
#load in the Twins markers

#healhy
healthy_markers1 <- read.csv('~/projects/TWINS_August_2021/GitHub/outs/MS_vs_Healthy_NON_pairwise/DGE_Healthy.csv')
healthy_markers2 <- read.csv('~/projects/TWINS_August_2021/GitHub/outs/SCNI_vs_Healthy_NON_pairwise/DGE_Healthy.csv')

#SCNI
scni_markers1 <- read.csv('~/projects/TWINS_August_2021/GitHub/outs/SCNI_vs_Healthy_NON_pairwise/DGE_SCNI.csv')
scni_markers2 <- read.csv('~/projects/TWINS_August_2021/GitHub/outs/SCNI_vs_MS_NON_pairwise/DGE_SCNI.csv')
#MS
ms_markers1 <- read.csv('~/projects/TWINS_August_2021/GitHub/outs/MS_vs_Healthy_NON_pairwise/DGE_MS.csv')
ms_markers2 <- read.csv('~/projects/TWINS_August_2021/GitHub/outs/SCNI_vs_MS_NON_pairwise/DGE_MS.csv')

#combine the markers
healthy_markers <- c(healthy_markers1$genes, healthy_markers2$genes)
healthy_markers <- unique(healthy_markers)

scni_markers <- c(scni_markers1$genes, scni_markers2$genes)
scni_markers <- unique(scni_markers)
ms_markers <- c(ms_markers1$genes, ms_markers2$genes)
ms_markers <- unique(ms_markers)

markers_twins <- c(scni_markers, ms_markers)
markers_twins <- unique(markers_twins)

In [ ]:
#save the volcano withouth highlighting

number_of_genes_volcano <- 40
options(repr.plot.width=10, repr.plot.height=8)
selected_markers_partner1$partner <- partner1
write.csv(selected_markers_partner1, file = paste0(dir_path, '/DGE_', partner1, '.csv'))
selected_markers_partner2$partner <- partner2
write.csv(selected_markers_partner2, file = paste0(dir_path, '/DGE_', partner2, '.csv'))

selected_markers_partner2$avg_log2FC <- -selected_markers_partner2$avg_log2FC 
selected_markers_combined <- rbind(selected_markers_partner1, selected_markers_partner2)


volcano <- ggplot(selected_markers_combined, aes(x = avg_log2FC, y = -log10(avg_p_val_adj))) +
        geom_vline(xintercept = 0) +
        geom_hline(yintercept = -log10(0.05), color ="grey", linetype ="dashed") +
        geom_point(data = selected_markers_combined,
                    color = "grey", alpha = 1) +
        geom_point(data = selected_markers_partner1[1:number_of_genes_volcano, ],
                    fill = colour1, alpha = 1, shape=21, size= 2.5) +
        geom_point(data = selected_markers_partner2[1:number_of_genes_volcano, ],
                    fill = colour2, alpha = 1, shape=21, size= 2.5) +
        geom_text_repel(data= rbind(selected_markers_partner1[1:number_of_genes_volcano, ], selected_markers_partner2[1:number_of_genes_volcano, ]), max.overlaps = number_of_genes_volcano, aes(label = genes))+
        theme_linedraw() +
        theme(panel.grid = element_blank(), legend.position = "none") +
        xlab("log2(average fold change)") +
        ylab("-log10(p-value)") + ggtitle(paste(partner2, '(left)', 'vs', partner1, '(right)'))
ggsave(volcano, file = paste0(dir_path, '/', partner1, '_vs_', partner2, '_volcano_plot.pdf'), height = 8, width = 10)

write.csv(selected_markers_combined, file = paste0(dir_path, '/DGE_', partner1, '_vs_', partner2, '.csv'))

In [ ]:
#save the volcano with highlighting

number_of_genes_volcano <- 40
highlight_color <- '#b61111'
options(repr.plot.width=10, repr.plot.height=8)

#subset the first lines from the object
selected_markers_partner1_number <- selected_markers_partner1[1:number_of_genes_volcano, ]
selected_markers_partner2_number <- selected_markers_partner2[1:number_of_genes_volcano, ]

# load in the non-twin cohort markers and leave only the ones were found in the twin cohort
ms_markers <- selected_markers_partner1
ms_markers <- ms_markers[ms_markers$genes %in% markers_twins, ]

IIH_markers <- selected_markers_partner2
IIH_markers <- IIH_markers[IIH_markers$genes %in% healthy_markers, ]

nrow(IIH_markers)
nrow(ms_markers)

text_all_top <- rbind(selected_markers_partner1[1:number_of_genes_volcano, ], selected_markers_partner2[1:number_of_genes_volcano, ])
text_all_top <- text_all_top[!text_all_top$genes %in% IIH_markers$genes, ]
text_all_top <- text_all_top[!text_all_top$genes %in% ms_markers$genes, ]

#leave only the markers that are in the top number_of_genes_volcano
highlight_MS <- ms_markers[ms_markers$genes %in% selected_markers_partner1_number$genes, ]
highlight_IIH <- IIH_markers[IIH_markers$genes %in% selected_markers_partner2_number$genes, ]


volcano <- ggplot(selected_markers_combined, aes(x = avg_log2FC, y = -log10(avg_p_val_adj))) +
        geom_vline(xintercept = 0) +
        geom_hline(yintercept = -log10(0.05), color ="grey", linetype ="dashed") +
        geom_point(data = selected_markers_combined,
                    color = "grey", alpha = 1) +
        geom_point(data = selected_markers_partner1[1:number_of_genes_volcano, ],
                    fill = colour1, alpha = 1, shape=21, size= 2.5) +
        geom_point(data = selected_markers_partner2[1:number_of_genes_volcano, ],
                    fill = colour2, alpha = 1, shape=21, size= 2.5) +
        geom_text_repel(data= rbind(highlight_MS, highlight_IIH), max.overlaps = number_of_genes_volcano, aes(label = genes), colour = highlight_color)+
        geom_text_repel(data= text_all_top, max.overlaps = number_of_genes_volcano, aes(label = genes))+
        theme_linedraw() +
        theme(panel.grid = element_blank(), legend.position = "none") +
        xlab("log2(average fold change)") +
        ylab("-log10(p-value)") + ggtitle(paste(partner2, '(left)', 'vs', partner1, '(right)'))
ggsave(volcano, file = paste0(dir_path, '/', partner1, '_vs_', partner2, '_volcano_plot_highlighted.pdf'), height = 8, width = 10)


# Create the UMAPs with immunological signatures (validation dataset)

In [ ]:
set.seed(1234)

In [ ]:
data <- read.csv2(file = './modules/immunological_markers_plot.csv')
data <- data$Gene

data <- data[data %in% combined_dis_np]
data <- data[order(data)]
data
length(data)
twins_immunological <- data
twins_immunological

In [ ]:
object_val <- readRDS(file = './mapped_pbmc_withoverlapdata_20220415.rds')
object_val <- NormalizeData(object_val, normalization.method = "LogNormalize", scale.factor = 10000)
object_val

#cluster of interest
clusters_of_interest <- c('6_GZMK', '8_CD74', '9_IKZF2', '10_FGFBP2')

#subset the desired clusters
object_val <- subset(object_val, idents = clusters_of_interest)

In [ ]:
unique(object_val@meta.data$cd8_coded)

In [ ]:
# load in the markers from the validation analysis
markers_val <- read.csv(file = './outs/MS_vs_IIH_NON_pairwise/DGE_MS.csv')
markers_val <- markers_val$genes

#map on the twins dataset
markers_val <- markers_val[markers_val %in% twins_immunological]
markers_val
length(markers_val)

In [ ]:
#create average expression object for the heatmaps
object_val@meta.data$diagnosis_expand <- paste0(object_val@meta.data$diagnosis_simp, '_', object_val@meta.data$expand)
Idents(object_val) <- 'diagnosis_expand'
levels(object_val) <- c('IIH_nonexp', 'IIH_exp', 'MS_nonexp', 'MS_exp')
object_av <- AverageExpression(object_val, return.seurat = T, verbose = FALSE)

In [ ]:
dir_plots <- './outs/signatures/'
colours_diagnosis_groups <- c('#1D5B60', '#1D5B60', '#8D2413', '#8D2413')
group_intereset <- 'MS_exp'
#run the loop for plotting new heamaps
    
    #create the table with expression values
    markers <- markers_val
    object_plot <- object_av
    
    #create margins for heatmap color scale
    data_markers <- object_av@assays$SymphonyQuery@scale.data
    data_markers <- data_markers[markers, ]
    max.value <- max(data_markers)
    min.value <- min(data_markers)
    
    #order expression
    data_markers <- data.frame(data_markers)
    data_markers <- data_markers[order(data_markers[[group_intereset]], decreasing = F), ]
    markers <- rownames(data_markers)
    
    #create the heatmap
    width <- 5.5
    height <- length(markers)/2
    options(repr.plot.width=width, repr.plot.height=height)
    
    #plot the heatmap
    heatmap <- DoHeatmap(object_plot, features = markers, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups) + 
            theme(text = element_text(size = 20, face = "plain", colour = 'black'),
                  axis.text.y=element_text(colour="black", size = 15)) + 
                  scale_fill_gradientn(colours = c("#2881C1", "white", "#D3556E", "#671727"), values = scales::rescale(c(min.value, 0, max.value/2, max.value)))
    heatmap
    ggsave(heatmap, file = paste0(dir_plots, 'immunological_mapped_heatmap_new_horizontal.pdf'), width = width, height = height)

In [ ]:
Idents(object_val) <- 'diagnosis_simp'
set.seed(1234)
number_downsample <- nrow(filter(object_val@meta.data, diagnosis_simp == 'IIH'))
object_subset <- subset(object_val, downsample = number_downsample)
table(object_subset$diagnosis_simp)
object_subset

In [ ]:
#check the number of cells per patient in the downsampled object
unique(filter(object_subset@meta.data, diagnosis_simp == 'IIH')$samplenumb)
unique(filter(object_subset@meta.data, diagnosis_simp == 'MS')$samplenumb)
table(filter(object_subset@meta.data, diagnosis_simp == 'MS')$samplenumb)

#original object in the MS group
table(filter(object_val@meta.data, diagnosis_simp == 'MS')$samplenumb)

In [ ]:
dir_plots <- './outs/signatures/'
    markers <- markers
    object_plot <- object_subset
    
    expression <- data.frame(matrix(NA, ncol = length(markers), nrow  = nrow(object_plot@meta.data)))
    for(i in 1:ncol(expression)){
        expression[, i] <- object_plot@assays$SymphonyQuery@data[markers[i], ]
        }
    
    expression[ncol(expression)+1] <- NA
    colnames(expression)[ncol(expression)] <- 'sum'
    
    for(i in 1:nrow(expression)){
        #with sum    
        expression$sum[i] <- sum(expression[i, 1:(ncol(expression)-1)]) / (ncol(expression)-1)
        #with geometrical mean
        #expression$sum[i] <- gm_mean(expression[i, 1:(ncol(expression)-1)])
    }
    
    object_plot@meta.data$sum_genes <- expression$sum
  
    #define the cells above the threshhold to plot
    cells_to_highlight <- rownames(object_plot@meta.data)[object_plot@meta.data$sum_genes > quantile(object_plot$sum_genes, 0.99)[[1]]]
    
    #plot the cells by the splitted column with selected threshhold
    object_split <- object_plot
    Idents(object_split) <- 'cd8_coded'
    width <- 11
    height <- 6
    options(repr.plot.width=12, repr.plot.height=6)
    object_split$diagnosis_simp <- factor(x = object_split$diagnosis_simp, levels = c('IIH', 'MS'))
    umap_plot <- DimPlot(object_split, reduction = 'umap', pt.size = 9.5, label = TRUE, repel  = TRUE, label.size = 6, cells.highlight = cells_to_highlight, split.by = 'diagnosis_simp', cols.highlight = '#D3556E', raster = T, raster.dpi = c(2048, 2048)) + 
    theme(
          text = element_text(size = 20),
          axis.text = element_text(size = 20),
          legend.text=element_text(size=20)) + NoLegend() + NoAxes()
    umap_plot
    ggsave(umap_plot, file = paste0(dir_plots, 'immunolgical_mapped_umap_featureplot.pdf'), width = width, height = height)
    
    umap_plot <- DimPlot(object_split, reduction = 'umap', label = TRUE, repel  = TRUE, label.size = 6, cells.highlight = cells_to_highlight, split.by = 'diagnosis_simp', cols.highlight = '#D3556E') + 
    theme(
          text = element_text(size = 20),
          axis.text = element_text(size = 20),
          legend.text=element_text(size=20)) + NoLegend() + NoAxes()

    ggsave(umap_plot, file = paste0(dir_plots, 'immunolgical_mapped_umap_featureplot_vector.pdf'), width = width, height = height)

# Create the UMAPs with metabolic signatures (validation dataset)

In [ ]:
set.seed(1234)

In [ ]:
object_val <- readRDS(file = './mapped_pbmc_withoverlapdata_20220415.rds')
object_val <- NormalizeData(object_val, normalization.method = "LogNormalize", scale.factor = 10000)
object_val

#cluster of interest
clusters_of_interest <- c('6_GZMK', '8_CD74', '9_IKZF2', '10_FGFBP2')

#subset the desired clusters
object_val <- subset(object_val, idents = clusters_of_interest)

In [ ]:
unique(object_val@meta.data$cd8_coded)

In [ ]:
data <- read.csv2(file = './modules/metabolic_markers_plot.csv')
data <- data$Gene
data <- unique(data)

data <- data[data %in% combined_dis_np]
data <- data[data %in% rownames(pbmc)]
data <- data[order(data)]
data
length(data)
twins_metabolic <- data

In [ ]:
# load in the markers from the validation analysis
markers_val <- read.csv(file = './outs/MS_vs_IIH_NON_pairwise/DGE_MS.csv')
markers_val <- markers_val$genes

#map on the twins dataset
markers_val <- markers_val[markers_val %in% twins_metabolic]
markers_val
length(markers_val)

In [ ]:
#create average expression object for the heatmaps
object_val@meta.data$diagnosis_expand <- paste0(object_val@meta.data$diagnosis_simp, '_', object_val@meta.data$expand)
Idents(object_val) <- 'diagnosis_expand'
levels(object_val) <- c('IIH_nonexp', 'IIH_exp', 'MS_nonexp', 'MS_exp')
object_av <- AverageExpression(object_val, return.seurat = T, verbose = FALSE)

In [ ]:
dir_plots <- './outs/signatures/'
colours_diagnosis_groups <- c('#1D5B60', '#1D5B60', '#8D2413', '#8D2413')
group_intereset <- 'MS_exp'
#run the loop for plotting new heamaps
    
    #create the table with expression values
    markers <- markers_val
    object_plot <- object_av
    
    #create margins for heatmap color scale
    data_markers <- object_av@assays$SymphonyQuery@scale.data
    data_markers <- data_markers[markers, ]
    max.value <- max(data_markers)
    min.value <- min(data_markers)
    
    #order expression
    data_markers <- data.frame(data_markers)
    data_markers <- data_markers[order(data_markers[[group_intereset]], decreasing = F), ]
    markers <- rownames(data_markers)
    
    #create the heatmap
    width <- 5.5
    height <- length(markers)/2
    options(repr.plot.width=width, repr.plot.height=height)
    
    #plot the heatmap
    heatmap <- DoHeatmap(object_plot, features = markers, draw.lines = FALSE, size = 7, raster = FALSE, group.colors = colours_diagnosis_groups) + 
            theme(text = element_text(size = 20, face = "plain", colour = 'black'),
                  axis.text.y=element_text(colour="black", size = 15)) + 
                  scale_fill_gradientn(colours = c("#2881C1", "white", "#D3556E", "#671727"), values = scales::rescale(c(min.value, 0, max.value/2, max.value)))
    heatmap
    ggsave(heatmap, file = paste0(dir_plots, 'metabolic_mapped_heatmap_new_horizontal.pdf'), width = width, height = height)

In [ ]:
Idents(object_val) <- 'diagnosis_simp'
set.seed(1234)
number_downsample <- nrow(filter(object_val@meta.data, diagnosis_simp == 'IIH'))
object_subset <- subset(object_val, downsample = number_downsample)
table(object_subset$diagnosis_simp)
object_subset

In [ ]:
#check the number of cells per patient in the downsampled object
unique(filter(object_subset@meta.data, diagnosis_simp == 'IIH')$samplenumb)
unique(filter(object_subset@meta.data, diagnosis_simp == 'MS')$samplenumb)
table(filter(object_subset@meta.data, diagnosis_simp == 'MS')$samplenumb)

#original object in the MS group
table(filter(object_val@meta.data, diagnosis_simp == 'MS')$samplenumb)

In [ ]:
dir_plots <- './outs/signatures/'
    markers <- markers
    object_plot <- object_subset
    
    expression <- data.frame(matrix(NA, ncol = length(markers), nrow  = nrow(object_plot@meta.data)))
    for(i in 1:ncol(expression)){
        expression[, i] <- object_plot@assays$SymphonyQuery@data[markers[i], ]
        }
    
    expression[ncol(expression)+1] <- NA
    colnames(expression)[ncol(expression)] <- 'sum'
    
    for(i in 1:nrow(expression)){
        #with sum    
        expression$sum[i] <- sum(expression[i, 1:(ncol(expression)-1)]) / (ncol(expression)-1)
        #with geometrical mean
        #expression$sum[i] <- gm_mean(expression[i, 1:(ncol(expression)-1)])
    }
    
    object_plot@meta.data$sum_genes <- expression$sum
  
    #define the cells above the threshhold to plot
    cells_to_highlight <- rownames(object_plot@meta.data)[object_plot@meta.data$sum_genes > quantile(object_plot$sum_genes, 0.99)[[1]]]
    
    #plot the cells by the splitted column with selected threshhold
    object_split <- object_plot
    Idents(object_split) <- 'cd8_coded'
    width <- 11
    height <- 6
    options(repr.plot.width=12, repr.plot.height=6)
    object_split$diagnosis_simp <- factor(x = object_split$diagnosis_simp, levels = c('IIH', 'MS'))
    umap_plot <- DimPlot(object_split, reduction = 'umap', pt.size = 9.5, label = TRUE, repel  = TRUE, label.size = 6, cells.highlight = cells_to_highlight, split.by = 'diagnosis_simp', cols.highlight = '#D3556E', raster = T, raster.dpi = c(2048, 2048)) + 
    theme(
          text = element_text(size = 20),
          axis.text = element_text(size = 20),
          legend.text=element_text(size=20)) + NoAxes() + NoLegend()
    umap_plot
    ggsave(umap_plot, file = paste0(dir_plots, 'metabolic_mapped_umap_featureplot.pdf'), width = width, height = height)
    
    umap_plot <- DimPlot(object_split, reduction = 'umap', label = TRUE, repel  = TRUE, label.size = 6, cells.highlight = cells_to_highlight, split.by = 'diagnosis_simp', cols.highlight = '#D3556E') + 
    theme(
          text = element_text(size = 20),
          axis.text = element_text(size = 20),
          legend.text=element_text(size=20)) + NoAxes() + NoLegend()

    ggsave(umap_plot, file = paste0(dir_plots, 'metabolic_mapped_umap_featureplot_vector.pdf'), width = width, height = height)